# Semantic Parsing and Text-to-SQL with Recurrent Neural Networks

In [0]:
__instructor__ = "Dragomir Radev"
__authors__ = "Tao Yu", "Angus Fong", "Suyi Li", "Alexander R. Fabbri", "Chris Hidey"
__version__ = "Introduction to Natural Language Processing, Spring 2019"

## Contents

0. Overview
0. Environment Setup
0. Introduction to NLP with Deep Learning
0. Assignment Part 1: Introduction to Pytorch and Semantic Parsing
0. Assignment Part 2: SQLNet for Text-to-SQL in Pytorch
0. Assignment Part 3: SQLNet with Torchtext
0. Submission

## Overview

In this assignment, You will:

* Learn to use some helpful tools such as Google Colab, Pytorch, Torchtext
* Develop a neural model for semantic parsing and the text-to-SQL Spider task

In order to make sure you understand every step in common NLP tasks, we provide some useful background information and show most of the code with detailed comments in this notebook. Thus, this assignment looks long, but you can skip some parts based on your own background (see Quick Search Tags below). Also, the coding part of this assignment should not be hard since we give you some examples to follow. The main idea of this assignment is to teach you about the entire pipeline for NLP with deep learning.

__Sections of this document__

Section A: Background

1. Introduction to Google Colab
2. Introduction to Semantic Parsing
3. Introduction to Pytorch

Section B: Implementation of Deep Learning for Semantic Parsing

4. Implementing SQLNet for Text-to-SQL in Pytorch
5. Implementing SQLNet with Torchtext

__Grading Criteria__

*  Short questions about basic concepts in Pytorch and the Spider task - ___20 points___
    0. Basic concepts in Pytorch - ___15 points___
    0. Write SQL in Spider task  - ___5 points___
    
* Implement SQLNet for Text-to-SQL in Pytorch - ___45 points___
    0. Implement the orderby module in SQLNet – __15__ points
    0. Compute the loss for the orderby module – __10__ points
    0. Compute accuracy for the orderby module – __10__ points
    0. Limitations of the SQLNet model on the Spider task – __10__ points
    
* Convert input pipeline for SQLNet using Torchtext - ___35 points___
    0. Reconstruct inputs for SQLNet – __15__ points
    0. Implement training process using Torchtext iterator – __20__ points
    
    
 __Quick Search Tags__
 
* **TODO**: to help you find questions and code snippets you have to address.

* We show all code on this notebook and add detailed comments for your quick reference. It can be too long to keep track of what is going on here, but don't be afraid! Please follow the tags below to know which parts you can just scan and which parts you need to read carefully.
  0. **Note**: very important things you should pay attention to
  0. **SCAN**: parts you can just quickly scan
  0. **#READCODE**: important code functions you need to understand
  0. need to read otherwise 
 
 **Note:** Do NOT remove any these Quick Search Tags in this notebook, TAs will use them to quickly find your answers.

## Environment Setup

For this assignment, we will use Google Colab. It is a free cloud service based on Jupyter Notebooks that lets you use a free GPU. Basically, you can quickly create, upload, share, and even edit togther Jupyter notebooks.

Please refer to [tutorial 1](https://course.fast.ai/start_colab.html) or [tutorial 2](https://towardsdatascience.com/getting-started-with-google-colab-f2fff97f594c) for additional instructions on how to use Google Colab.

#### Using GPU in Colab
Before running anything, you need to tell Colab that you want to use a GPU: 
1. Go to __Runtime__ option on the top left
2. Click __Change runtime type__
3. Select "Python 3" for __Runtime type__ and "GPU" for __Hardward accelerator__
4. Click __SAVE__ button

Colab has popular libraries already installed such as Pytorch, TensorFlow, OpenCV and Keras. Let's get started and verify this:

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Pytorch version is: ", torch.__version__)
print("You are using: ", device)

Pytorch version is:  1.0.1.post2
You are using:  cuda


You should be using CUDA with Pytorch 1.0.1.post2.

**Note**: Of course, there are some limits. For example, it allows you access to a free GPU for only 10 hours at one time.

**Note**:  Also, your downloaded files will NOT persist after the allocated instance is shutdown. To avoid this, you need to permit Colab instance to read and write files to your Google Drive. Use the following code snippet to mount your Google Drive:

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


You need to click the link, copy the code on the page, paste it in the box, hit enter, and you’ll see the message "Mounted at /content/gdrive" when you’ve successfully mounted your drive. 

**TODO**: Let's download [`nlp_hw4`](https://drive.google.com/drive/folders/1_a_01EZfCUnuWnchEfRHZnLsWIYwka_k?usp=sharing) (unzip it), which contains the data and files which will be used in this assignment, and put it on your Google Drive. Set directory paths below:

In [0]:
import os
ROOT_DIR = "/content/gdrive/My Drive/nlp_hw4/"
DATA_DIR = os.path.join(ROOT_DIR, "data")
GLOVE_DIR = os.path.join(ROOT_DIR, "glove.6B.50d.txt")
SAVED_MODEL_DIR = os.path.join(ROOT_DIR, "saved_models")
TABLE_PATH = os.path.join(DATA_DIR, "tables.json")
TRAIN_PATH = os.path.join(DATA_DIR, "train_spider.json")
DEV_PATH = os.path.join(DATA_DIR, "dev.json")

Again, please refer to [tutorial 1](https://course.fast.ai/start_colab.html) or [tutorial 2](https://towardsdatascience.com/getting-started-with-google-colab-f2fff97f594c) for more details on how to use Google Colab.

 **Note**: You need to finish this assignment on Google Colab  because you will share your Google Colab notebook with TAs for grading.
 
 You should be able to import all packages below:

In [0]:
import re
import io
import sys
import tqdm
import json
import datetime
import argparse
import numpy as np
from itertools import chain, count
from collections import Counter, defaultdict

import torch
import torchtext.data
import torchtext.vocab
import torch.nn as nn
import torch.nn.functional as F

sys.path.append(ROOT_DIR)
from utils import SelPredictor, GroupPredictor, lower_keys, to_batch_query

## Introduction to NLP with Deep Learning (Optional)

**Note**: This section contains some useful links and resources on NLP with Deep Learning  for your reference in the future. For this assignment, we assume you already understand these concepts. You can **SCAN** this part if you want. 

Deep learning is currently a very popular topic in NLP. Deep learning approaches have very recently obtained state-of-the-art performance across many different NLP tasks. The appeal of deep learning is enhanced by the idea that tasks can be modeled without extensive feature engineering. The term “deep learning“ often refers to neural networks with many hidden layers. However, many people use the term for any neural network with non-linear transformations.

The standard multi-layer neural network (also called [a multi-layer perceptron or MLP](http://deeplearning.net/tutorial/mlp.html)) looks something like this:


<center><img src="https://i.ibb.co/7knDjRp/mlp.png" alt="mlp" align="middle"></center>


In this example, the input layer $x$ is of size $d_0=3$, the hidden layer $h$ is of size $d_1=4$, and the output layer $f(x)$ is of size $d_2=2$. In matrix notation:

\begin{equation*}
f(x) = g_2 \left( W_2 \cdot g_1(W_1 \cdot x + b_1) + b_2 \right)
\end{equation*}

where $W_n\in R^{d_n \times d_{n-1}}$ and $b_n\in R^{d_n}$ are the __parameters__ of the network for layer $n$ and $g_n$ is an __activation function__ for layer $n$. We learn the parameters by minimizing a __cost function__ using some __optimization method__. The __hyperparameters__ of the model are the dimensionality of the input and hidden layers and number of hidden nodes.

Determining the network structure, activation function, cost function, and optimization method is part of the process of modeling a neural network. If the cost function is differentiable, we can take the gradient of the cost function with respect to the parameters and update the parameters using backpropagation (backpropagation is a very important step to build a good network model, [here](http://colah.github.io/posts/2015-08-Backprop/) is a nice, simple tutorial on backpropagation) and apply a gradient-based optimization technique. For this course, we will not be deriving gradients but rather using a library for [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) (for a better understanding of deep learning you should derive and implement backpropagation rather than using an autograd library). Alternatively, there are many search-based algorithms for learning neural network parameters without differentiation.

There are many gradient-based optimization algorithms (popular methods include [Adagrad](http://sebastianruder.com/optimizing-gradient-descent/index.html#adagrad), [Adam](http://sebastianruder.com/optimizing-gradient-descent/index.html#adam), and [RMSprop](http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop). Refer to [Wiki](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) or [here](http://sebastianruder.com/optimizing-gradient-descent/index.html#gradientdescentoptimizationalgorithms) for more details). [Adagrad](http://sebastianruder.com/optimizing-gradient-descent/index.html#adagrad) is the method used for this project and is an improved form of gradient descent. If you have taken machine learning, you are probably familiar with gradient descent. [Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) is a basic optimization algorithm for finding the minimum of a function.

In order to actually model a network, we still need to select an activation function and a cost function. An [activation function](https://en.wikipedia.org/wiki/Activation_function) is a non-linear function that allows the MLP to approximate any function. Common activation functions include the [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function), [hyperbolic tangent](https://en.wikipedia.org/wiki/Hyperbolic_function#Hyperbolic_tangent), and [rectified linear unit](https://en.wikipedia.org/wiki/Rectifier_(neural_networks) (ReLU or rectifier). The cost function is very task-specific. We may wish to minimize squared error or another difference metric. We may also want a probabilistic interpretation. In this framework we want to minimize the negative log likelihood of the training data. One common probabilistic cost function is the [softmax function](https://en.wikipedia.org/wiki/Softmax_function).

Two common network structures in NLP are [recurrent](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) and [recursive](http://www.iro.umontreal.ca/~bengioy/talks/gss2012-YB6-NLP-recursive.pdf) neural networks. Recurrent neural networks are used for sequential input or time series. These networks have been used successfully in NLP for part-of-speech tagging and named entity recognition. Recursive neural networks are used for tree structures and have had success at parsing and sentiment analysis. The difference between these networks and the MLP is that they model “hidden states” at the current time or node that are a composition of all previous states. Previously in this class we studied HMMs, where the hidden states are discrete variables. In the recurrent neural network, the hidden state is a continuous variable.

Here is a really good, vivid diagram of a recurrent neural network from [Stanford cs224u](http://nbviewer.jupyter.org/github/cgpotts/cs224u/blob/master/nli.ipynb#Homework-4) (It shows many critical steps and implementation details in training any neural networks (not just recurrent ones) in NLP. This graph is very helpful in fully understanding this assignment.):

<center><img src="https://i.ibb.co/NTCDZ7f/rnn.png" alt="rnn" width="450"></center>

The model definition of the above graph is as follows:

\begin{equation*}
h_t = g_h \left( W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h \right)
\end{equation*}

\begin{equation*}
f(x_t) = g_o \left( h_t \cdot W_{hy} + b_o \right)
\end{equation*}

where each hidden state $h_t$ is a composition of the previous state and $x_t$, the feature vector for element $x$ at time $t$. The parameters to be learned are the weight matrices $W$, the biases $b$, and an initial state $h_0$. For a specific application, consider that the elements $x_t$ are words and the outputs $f(x_t)$ are part-of-speech tags. We may choose the sigmoid function for $g_h$ and the softmax function for $g_o$ which will give us a probability distribution for a word at time $t$.

In many NLP applications, the input vector will be a __word embedding__ (which we learned about in the previous assignments), a continuous representation of a word. These word embeddings may be pre-trained using methods such as [_GloVe_](http://nlp.stanford.edu/projects/glove/) or [_Word2vec_](https://code.google.com/archive/p/word2vec/) and then fed into the model. During training, there are several variations on how we treat word embeddings:

0. The embeddings may be fixed and not changed during training.
0. We may initialize them with pre-trained embeddings and learn them as parameters (possibly also adding a constraint that they are not allowed to vary too much).
0. We may not use pre-trained embeddings at all and learn them as parameters with random initialization. 

Generally, the ad hoc wisdom is that for small-ish datasets it is better to use fixed embeddings.

__Additional Resources__

Many would still be confused by deep learning after this tutorial (it's really hard to understand!). Becuase this class is about NLP instead of deep learning, we can't provide a long tutorial here. You can check out the following materials for an in-depth understanding:

* The links of key words in the above sections: most of them have nice tutorials with vivid examples and graphs instead of just theoretical interpretations.
* [Great intro (with vivid diagrams) to LSTM/backpropagation/other ideas in networks](http://colah.github.io/)
* [Using pre-trained word embeddings in networks](http://sebastianruder.com/word-embeddings-1/index.html)
* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* For a full exploration of deep learning, check out NLP with deep learning classes at [Stanford](http://web.stanford.edu/class/cs224n/index.html) and [CMU](http://www.phontron.com/class/nn4nlp2019/).

## Assignment Part 1: Introduction to Pytorch and Semantic Parsing - 20 Points

__a. Concepts and  Questions on Pytorch__

We suppose that you are already familiar with basic concepts of Pytorch (used Pytorch in homework 2). At its core, PyTorch provides two main features: an n-dimensional Tensor, similar to Numpy but canrun on GPUs, and automatic differentiation for building and training neural networks. 

Please check out more details in [the Pytorch official tutorial](https://pytorch.org/tutorials/index.html). Especially, the following toturials are most helpful:

* [Deep Learning with PyTorch: A 60 Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
* [Learning PyTorch with Examples](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)
* [What is torch.nn really?](https://pytorch.org/tutorials/beginner/nn_tutorial.html)
* [Deep Learning for NLP with Pytorch](https://pytorch.org/tutorials/beginner/deep_learning_nlp_tutorial.html)
* [Chatbot Tutorial](https://pytorch.org/tutorials/beginner/chatbot_tutorial.html)


**Note**: To make this learning process even better, we put the full example code of all tutorials on the Colab. You can just copy [the notebooks](https://drive.google.com/drive/folders/1ltvDFeqMbV9FxnUfMWyD83X6iwfmjCWS?usp=sharing) to your directory and run the code directly on Colab!

**TODO: Please answer the questions below (refer to the official tutorials) - 15 points**:
1. Why do we run `model.eval()` during the evaluation?<br/>
 **YOUR ANSWER:** By default all modules in PyTorch are initialized to train mode, `model.eval()` explicitely indicates that code is switched to evaluation mode. Note that some layers may have different behaviors in train mode vs. evaluation mode, for example, bacth norm, or dropout.
2. Why do we run `optimizer.zero_grad()` during the training?<br/>
 **YOUR ANSWER:** PyTorch by default accumulates gradients on subsequent backward passes. This is useful when training RNN, but in some other occasions where we don't want to accumulate the gradients (e.g., training a regular feed forward NN with mini-batches), `optimizer.zero_grad()` should be run during training.
3. What do `loss.backward()` and `optimizer.step()` do?<br/>
 **YOUR ANSWER:** `loss.backward()` computes the gradients (of the loss w.r.t. the parameters), `optimizer.step()` performs the gradient descent (to actually update the parameters).

__b. Introduction to Semantic Parsing and Text-to-SQL Spider Task – 5 points__

___Semantic Parsing___ aims to map nautral language questions to executable programs such as logical forms, Python code, and SQL queries. The text-to-SQL task is one of the most important subtasks of semantic parsing. It converts natural language sentences to corresponding [SQL queries](https://www.w3schools.com/sql/sql_intro.asp).

For exmaple, the goal of a text-to-SQL system is to automatically convert  ___"What are the name and budget of the departments with average instructor salary greater than the overall average?"___ into the SQL query below:

```
SELECT T2.name, T2.budget
FROM instructor as T1 JOIN department as T2 ON T1.department_id = T2.id 
GROUP BY T1.department_id
HAVING avg(T1.salary) > 
    (SELECT avg(salary) FROM instructor)
 ```

**TODO**: Write a SQL query for "What is the name of the department with the highest average instructor salary?" – __5 points__<br/>
**YOUR ANSWER:** 
 
 ```
SELECT T2.name
FROM instructor as T1 JOIN department as T2 ON T1.department_id = T2.id
GROUP BY T1.department_id
ORDER BY avg(T1.salary) DESC
LIMIT 1
 ```

[___Spider___](https://yale-lily.github.io/spider) is a large-scale complex and cross-domain text-to-SQL challenge developed here at Yale. It has quickly become one of the most popular tasks in this filed. We already received several submissions from various research labs. The goal of the Spider challenge is to develop natural language interfaces to cross-domain databases. It consists of 10k+ question-SQL pairs querying about 200 databases. In Spider, different complex SQL queries and databases appear in train and test sets. To do well on this task, systems must generalize well to not only new SQL queries but also new database schemas. Read more from [our EMNLP paper](https://arxiv.org/pdf/1809.08887.pdf) and [Medium blog](https://medium.com/@tao.yu/spider-one-more-step-towards-natural-language-interfaces-to-databases-62298dc6df3c).

## Assignment Part 2: SQLNet for Text-to-SQL in Pytorch - 45 Points

In this assignment, we are going to implement a neural text-to-SQL model [SQLNet](https://arxiv.org/pdf/1711.04436.pdf) for the Spider challenge. SQLNet was originally developed for a simpler text-to-SQL [WikiSQL task](https://einstein.ai/static/images/pages/research/seq2sql/seq2sql.pdf) (SQL queries in the WikiSQL dataset only include `SELECT` and `WHERE` clauses). It divides SQL generation into two parts/modules filling slots in **`SELECT`** (`$AGG $COLUMN` of `SELECT` in SQL sketch shown below) and **`WHERE`** (`$COLUMN $OP` of `WHERE` in SQL sketch) clauses:

<center><img src="https://i.ibb.co/BGbcFNL/sqlnet-model.png" alt="rnn" width="450"></center>

We are going to extend SQLNet to the Spider task to include more complex `SELECT`, `GROUP BY` and `ORDER BY` modules.  Please read more about SQLNet model in [this paper](https://arxiv.org/pdf/1711.04436.pdf) (sepecially section 3.2 and 3.3).

As you may noticed on [the leaderboard](https://yale-lily.github.io/spider), the performance of SQLNet on the Spider task is only 12.4% because of the difficulty of the task and the model's limitations. There are many research institutions which are working on this task with recently-introduced models (including more sophisticated [SyntaxSQL](https://arxiv.org/pdf/1810.05237.pdf) model developed by [the LILY lab](https://yale-lily.github.io/)). The best submission so far can achieve around 50% accuarcy (we will add their results on the leaderboard only after their papers get published). However, SQLNet is still a good example model to introduce you to the field of semantic parsing.

In order to better understand the whole pipeline for common NLP tasks, you will be implementing many key steps including data preprocessing, a word embedding layer, batching, the key modules of SQLNet, loss functions, prediction postprocessing, and, finally, evaluation.

#### 1. Data Loading and Preprocessing

**Spider Data Content and Format**: Please read [the Spider Github page](https://github.com/taoyds/spider#data-content-and-format) to understand the Spider data content and format. Basically, there are two main raw inputs: the natural language questions in `train.json` and the database schema info (table and column names etc.) in `tables.json`. The gold labels are SQL queries. **Note**: In Spider task, we needn't to predict any values in SQL such as 42 in the example below (for reasons for this decision, please refer to the Spider paper).

<center><img src="https://i.ibb.co/87ZZfvt/sqlnet.png" alt="rnn" width="500"></center>

First, we need to load these data files and convert raw SQL queries into target labels for each module in SQLNet. **Note**: Please go over the code to understand the data input structures.



In [0]:
#READCODE!
def process(sql_path, table_data):
    '''
    process data into inputs for modules in SQLNet
    
    ---Parameters---
    
    sql_path (str): dir to a input file that contains questions and SQL queries 
            such as train.json or dev.json 
    table_data (list of dicts): each dict in the list contains schema info 
            (table and column names etc.) of each database
        
    ---Returns---

    output_sql (list of dicts): each dict in the list is a training data point 
            with question, database info, and target labels for each module 
            (agg/sel/SELECT, cond/WHERE, group/GROUP, order/ORDER) in SQLNet
    '''
    print("Loading data from %s"%sql_path)
    sql_data = []
    #read and lowercase text
    with open(sql_path) as inf:
        data = lower_keys(json.load(inf))
        sql_data += data
    
    #reformat table info
    tables = {}
    for i in range(len(table_data)):
        table = table_data[i]
        db_name = table['db_id']
        tables[db_name] = table

    #for each data point, store question and database info, and covert SQL into
    #label inputs for each module in SQLNet
    output_sql = []
    for i in range(len(sql_data)):
        sql = sql_data[i]
        sql_one = {}

        # add query metadata
        sql_one['question'] = sql['question']
        sql_one['question_tok'] = sql['question_toks']
        sql_one['query'] = sql['query']
        sql_one['query_tok'] = sql['query_toks']
        sql_one['table_id'] = sql['db_id']
        table = tables[sql['db_id']]
        sql_one['column_names'] = table['column_names']
        sql_one['col_org'] = table['column_names_original']
        sql_one['table_org'] = table['table_names_original']
        sql_one['fk_info'] = table['foreign_keys']
        
        # process aggregation/column selection labels for SELECT module
        # agg labels are indices of ('none', 'max', 'min', 'count', 'sum', 'avg')
        # sel labels are indices of column names
        sql_one['agg'] = []
        sql_one['sel'] = []
        gt_sel = sql['sql']['select'][1]
        if len(gt_sel) > 3: # limit to select at most 3 columns
            gt_sel = gt_sel[:3]
        for tup in gt_sel:
            sql_one['agg'].append(tup[0])
            sql_one['sel'].append(tup[1][1][1]) #GOLD for sel and agg
        
        # process where conditions
        # op labels are indices of ('not', 'between', '=', '>', '<', '>=', '<=', 
        # '!=', 'in', 'like', 'is', 'exists')
        sql_one['cond'] = []
        gt_cond = sql['sql']['where']
        if len(gt_cond) > 0:
            conds = [gt_cond[x] for x in range(len(gt_cond)) if x % 2 == 0]
            for cond in conds:
                curr_cond = []
                curr_cond.append(cond[2][1][1])
                curr_cond.append(cond[1])
                if cond[4] is not None:
                    curr_cond.append([cond[3], cond[4]])
                else:
                    curr_cond.append(cond[3])
                sql_one['cond'].append(curr_cond) #GOLD for COND [[col, op],[]]

        # process group by / having
        sql_one['group'] = [x[1] for x in sql['sql']['groupby']] #assume only one groupby
        having_cond = []
        if len(sql['sql']['having']) > 0:
            gt_having = sql['sql']['having'][0] # currently only do first having condition
            having_cond.append([gt_having[2][1][0]]) # aggregator
            having_cond.append([gt_having[2][1][1]]) # column
            having_cond.append([gt_having[1]]) # operator
            if gt_having[4] is not None:
                having_cond.append([gt_having[3], gt_having[4]])
            else:
                having_cond.append(gt_having[3])
        else:
            having_cond = [[], [], []]
        sql_one['group'].append(having_cond) #GOLD for GROUP [[col1, col2, [agg, col, op]], [col, []]]

        # process order by / limit
        order_aggs = []
        order_cols = []
        sql_one['order'] = []
        order_par = 4
        gt_order = sql['sql']['orderby']
        limit = sql['sql']['limit']
        if len(gt_order) > 0:
            order_aggs = [x[1][0] for x in gt_order[1][:1]] # limit to 1 order by
            order_cols = [x[1][1] for x in gt_order[1][:1]]
            if limit != None:
                if gt_order[0] == 'asc':
                    order_par = 0
                else:
                    order_par = 1
            else:
                if gt_order[0] == 'asc':
                    order_par = 2
                else:
                    order_par = 3

        sql_one['order'] = [order_aggs, order_cols, order_par] #GOLD for ORDER [[[agg], [col], [dat]], []]

        # ingore intersect/except/union and nested queries because of 
        # limitations of SQLNet
        
        output_sql.append(sql_one)
        
    return output_sql


Load and process train and development data. Now you can print some examples to see what the data looks like.



In [0]:
def load_dataset(dataset_dir):
    '''
    load and process train and dev datasets
    '''
    with open(TABLE_PATH) as inf:
        print("Loading data from %s"%TABLE_PATH)
        table_data= json.load(inf)
    
    train_sql_data = process(TRAIN_PATH, table_data)
    val_sql_data = process(DEV_PATH, table_data)

    return train_sql_data, val_sql_data

In [8]:
train_sql_data, val_sql_data = load_dataset(DATA_DIR)

Loading data from /content/gdrive/My Drive/nlp_hw4/data/tables.json
Loading data from /content/gdrive/My Drive/nlp_hw4/data/train_spider.json
Loading data from /content/gdrive/My Drive/nlp_hw4/data/dev.json


#### 2. Word Embedding Layer

Load pre-trained Glove word embedding `glove.6B.50d.txt`:

In [0]:
def load_word_emb(file_name):
    '''
    load and convert pretrained glove embedding
    
    ---Returns---
    ret (a dict): a embedding dict in which the key is a word and the value is 
          the embedding (np array) of that word.
    '''
    print ('Loading word embedding from %s'%file_name)
    ret = {}
    with open(file_name) as inf:
        for idx, line in enumerate(inf):
            info = line.strip().split(' ')
            if info[0].lower() not in ret:
                ret[info[0]] = np.array(list(map(lambda x:float(x), info[1:])))
    return ret

In [10]:
word_emb = load_word_emb(GLOVE_DIR)

Loading word embedding from /content/gdrive/My Drive/nlp_hw4/glove.6B.50d.txt


Build a word embedding layer containing functions that map words in questions and column names of databases into fixed-length embedding vectors.

In [0]:
#READCODE!
class WordEmbedding(nn.Module):
    '''
    word embedding layer mapping words in questions and column names into embeddings
    '''
    def __init__(self, word_emb, N_word):
        '''
        initialization method
        
        ---Parameters---
        
        word_emb (a dict): dict of loaded glove embeddings
        N_word (int): embedding dimension
        '''
        super(WordEmbedding, self).__init__()
        self.N_word = N_word
        self.word_emb = word_emb

    def gen_x_batch(self, q):
        '''
        map question inputs in each batch into embeddings
        
        ---Parameters---
        
        q (list of lists): each list include words in each question
        
        ---Returns---
        
        val_inp (tensor): question embedding input tensor for the model
        val_len (list): each value in the list is the length of each question
        '''
        B = len(q)
        val_embs = []
        val_len = np.zeros(B, dtype=np.int64)
        for i, one_q in enumerate(q):
            #map each word into embedding
            q_val = list(map(lambda x:self.word_emb.get(x, np.zeros(self.N_word, dtype=np.float32)), one_q))
            #add start and end special tokens
            val_embs.append([np.zeros(self.N_word, dtype=np.float32)] + q_val + [np.zeros(self.N_word, dtype=np.float32)])  #<BEG> and <END>
            val_len[i] = 1 + len(q_val) + 1 
        max_len = max(val_len)
        #embedding output size is (batch size, max word length, embedding dim)
        val_emb_array = np.zeros((B, max_len, self.N_word), dtype=np.float32)
        for i in range(B):
            for t in range(len(val_embs[i])):
                val_emb_array[i,t,:] = val_embs[i][t]
        val_inp = torch.from_numpy(val_emb_array).to(device)

        return val_inp, val_len


    def gen_col_batch(self, cols):
        '''
        map column names in each batch into embeddings
        
        ---Parameters---
        
        cols (list of lists of lists): list (batch of databases) of lists 
              (column names in each database) of lists (words in each column)
        
        ---Returns---
        
        name_inp (tensor): shape is (num of all column names in the batch, 
              embedding dim), each row in the tensor is embeddings of words 
              for each column name
        name_len (list): each value in the list is the number of words in each 
              column name
        col_len (list): each value in the list is the number of column names in 
              each database
        '''
        ret = []
        col_len = np.zeros(len(cols), dtype=np.int64)
        
        #flatten cols into a list of lists of all column names in the batch
        #[[list of cols[col1...] in one db], ...] -> [[col1], [col2],...]
        names = []
        for b, one_cols in enumerate(cols):
            names = names + one_cols
            col_len[b] = len(one_cols)
        #map all column names in the batch into embeddings
        name_inp, name_len = self.str_list_to_batch(names)
        
        return name_inp, name_len, col_len

      
    def str_list_to_batch(self, str_list):
        """
        get a list var of wemb of words in each column name in current bactch
        refer to gen_col_batch method
        """
        B = len(str_list)

        val_embs = []
        val_len = np.zeros(B, dtype=np.int64)
        for i, one_str in enumerate(str_list):
            val = [self.word_emb.get(x, np.zeros(
                  self.N_word, dtype=np.float32)) for x in one_str]
            val_embs.append(val)
            val_len[i] = len(val)
        max_len = max(val_len)

        val_emb_array = np.zeros(
                (B, max_len, self.N_word), dtype=np.float32)
        for i in range(B):
            for t in range(len(val_embs[i])):
                val_emb_array[i,t,:] = val_embs[i][t]
        val_inp = torch.from_numpy(val_emb_array).to(device)

        return val_inp, val_len

#### 3. Running LSTM

Below are functions to run an LSTM using packed sequence.

In [0]:
def run_lstm(lstm, inp, inp_len, hidden=None):
    '''
    run the LSTM using packed sequence
    
    ---Parameters---

    lstm (object): a Pytorch LSTM module
    inp (tensor): embedding input with shape (batch size, max x len, embed dim)
    inp_len (list): number of words in input sequences in current batch
    hidden (tensor): as initial hidden state

    ---Returns---

    ret_s (tensor): LSTM output containing the output features (h_t) from the 
          last layer of the LSTM, for each t
    ret_h (tuple of tensors): (h_n, c_n), h_n: hidden states, c_n: cell states
    '''
    #running LSTM with packed sequence requires to first sort the input 
    #according to its length.
    sort_perm = np.array(sorted(range(len(inp_len)),
        key=lambda k:inp_len[k], reverse=True))
    sort_inp_len = inp_len[sort_perm]
    sort_perm_inv = np.argsort(sort_perm)
    sort_perm = torch.LongTensor(sort_perm).to(device)
    sort_perm_inv = torch.LongTensor(sort_perm_inv).to(device)
    #reconstruct inp embedding input based on sorted indices
    #pack padded batch of sequences for LSTM module
    lstm_inp = nn.utils.rnn.pack_padded_sequence(inp[sort_perm],
            sort_inp_len, batch_first=True)
    if hidden is None:
        lstm_hidden = None
    else:
        lstm_hidden = (hidden[0][:, sort_perm], hidden[1][:, sort_perm])
    #run LSTM
    sort_ret_s, sort_ret_h = lstm(lstm_inp, lstm_hidden)
    #unpack a Tensor containing padded sequences of variable length.
    ret_s = nn.utils.rnn.pad_packed_sequence(
            sort_ret_s, batch_first=True)[0][sort_perm_inv]
    ret_h = (sort_ret_h[0][:, sort_perm_inv], sort_ret_h[1][:, sort_perm_inv])
    
    return ret_s, ret_h


def col_name_encode(name_inp_var, name_len, col_len, enc_lstm):
    '''
    run LSTM to encode the columns. The embedding of a column name is the last 
    state of its LSTM output.
    
    ---Parameters---

    name_inp_var (tensor): shape is (num of all column names in the batch, 
          embedding dim), each row in the tensor is embeddings of words 
          for each column name
    name_len (list): each value in the list is the number of words in each 
          column name
    col_len (list): each value in the list is the number of column names in 
          each database
    enc_lstm (object): a Pytorch LSTM module

    ---Returns---

    ret (tensor): LSTM output with shape (batch size, num of columns in each db,
          embedding dim)
    '''
    #run LSTM between words in each column name
    name_output, _ = run_lstm(enc_lstm, name_inp_var, name_len)
    name_out = name_output[tuple(range(len(name_len))), name_len-1]
    #shape: (batch size, num of columns in each db, embedding dim)
    ret = torch.zeros(len(col_len), max(col_len), name_out.size()[1], device=device)
    st = 0
    for idx, cur_len in enumerate(col_len):
        ret[idx, :cur_len] = name_out.data[st:st+cur_len]
        st += cur_len
        
    return ret

#### 4. Modules in SQLNet

As we mentioned, SQLNet employs independent modules to predict slots in `SELECT`,  `WHERE`,  `GROUPBY`, and `ORDERBY` clauses, and then combines their predictions together to generate the final predicted SQL query.

This section shows code for each module. Bascially, for each module, it does the following:
0. runs LSTM forward pass on words of the question.
0. runs LSTM forward pass on words of all column names in current batch.
0. computes attention values between each column name and words of the question and gets the final question representation weighted the attentions.
0. inputs the final question representation to nonlinearities to predict a probability distribution over all possible values for each slot in each clause.

In [0]:
#READCODE!
class CondPredictor(nn.Module):
    '''
    module to predict condition number, columns, and operators in WHERE condition.
    '''
    def __init__(self, N_word, N_h, N_depth):
        '''
        ---Parameters---
        
        N_word (int): embedding dimension
        N_h (int): hidden size
        N_depth (int): number of recurrent layers.
        '''
        super(CondPredictor, self).__init__()
        self.N_h = N_h #hidden size
        #initialize LSTM for encoding questions
        self.q_lstm = nn.LSTM(input_size=N_word, hidden_size=N_h//2,
                num_layers=N_depth, batch_first=True,
                dropout=0.3, bidirectional=True)
        #initialize LSTM for encoding column names
        self.col_lstm = nn.LSTM(input_size=N_word, hidden_size=N_h//2,
                num_layers=N_depth, batch_first=True,
                dropout=0.3, bidirectional=True)
        #initialize functions for condition number prediction
        #limit the number of conditions to 0-5
        self.q_num_att = nn.Linear(N_h, N_h)
        self.col_num_out_q = nn.Linear(N_h, N_h)
        self.col_num_out = nn.Sequential(nn.Tanh(), nn.Linear(N_h, 6))
        #initialize functions for condition column prediction
        self.q_att = nn.Linear(N_h, N_h)
        self.col_out_q = nn.Linear(N_h, N_h)
        self.col_out_c = nn.Linear(N_h, N_h)
        self.col_out = nn.Sequential(nn.Tanh(), nn.Linear(N_h, 1))
        #initialize functions for condition operation prediction
        #the final op out dimension 12 is probs over all possible operators below 
        #('not', 'between', '=', '>', '<', '>=', '<=', '!=', 'in', 'like', 'is', 'exists')
        self.op_att = nn.Linear(N_h, N_h)
        self.op_out_q = nn.Linear(N_h, N_h)
        self.op_out_c = nn.Linear(N_h, N_h)
        self.op_out = nn.Sequential(nn.Tanh(), nn.Linear(N_h, 12)) 

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q_emb_var, q_len, col_emb_var, col_len, col_name_len, gt_cond):
        '''
        forward pass for condition module
        
        ---Parameters---
        
        q_emb_var (tensor): embeddings for question inputs,
                shape: (batch size, max question len, embed dim)
        q_len (list): number of words in question inputs in current batch
        col_emb_var (tensor): embeddings for all column names in current batch,
                shape: (num of all column names in the batch, embed dim), each 
                row in the tensor is embeddings of words for each column name
        col_len (list): each value in the list is the number of column names in 
                each database
        col_name_len (list): each value in the list is the number of words in each 
                column name
                
        ---Returns---
        
        score (tuple): includes col_num_score (batch size, 4), 
                col_score (batch size, max_col_len), op_score (batch size, 5, 12)?
        '''
        max_q_len = max(q_len)
        max_col_len = max(col_len)
        B = len(q_len)
        #forward pass through LSTM for questions and column names
        q_enc, _ = run_lstm(self.q_lstm, q_emb_var, q_len)
        col_enc = col_name_encode(col_emb_var, col_name_len, col_len, self.col_lstm)

        ## Predict condition number: 0-5 ## 
        # att_val_qc_num: (B, max_col_len, max_q_len)
        # att_val_qc_num: attention before softmax tensor v btw q and col in SQLNet paper
        att_val_qc_num = torch.bmm(col_enc, self.q_num_att(q_enc).transpose(1, 2))
        #set prob for padded words to negative 100
        for idx, num in enumerate(col_len):
            if num < max_col_len:
                att_val_qc_num[idx, num:, :] = -100
        for idx, num in enumerate(q_len):
            if num < max_q_len:
                att_val_qc_num[idx, :, num:] = -100
        # att_prob_qc_num: attention tensor w in SQLNet paper
        att_prob_qc_num = self.softmax(att_val_qc_num.view((-1, max_q_len))).view(B, -1, max_q_len)
        # q_weighted_num: (B, hid_dim) modified E_Q/Q
        q_weighted_num = (q_enc.unsqueeze(1) * att_prob_qc_num.unsqueeze(3)).sum(2).sum(1)
        # self.col_num_out: (B, 6)
        # col_num_score: P_#col(K/Q) without softmax (because cros entropy loss 
        # in Pytorch does softmax) in SQLNet paper
        col_num_score = self.col_num_out(self.col_num_out_q(q_weighted_num))

        ## Predict columns in WHERE condition ## 
        # att_val_qc: attention before softmax tensor v btw q and col in SQLNet paper
        att_val_qc = torch.bmm(col_enc, self.q_att(q_enc).transpose(1, 2))
        for idx, num in enumerate(q_len):
            if num < max_q_len:
                att_val_qc[idx, :, num:] = -100
        # att_prob_qc: attention tensor w in SQLNet paper
        att_prob_qc = self.softmax(att_val_qc.view((-1, max_q_len))).view(B, -1, max_q_len)
        # q_weighted: (B, max_col_len, hid_dim)
        # q_weighted: E_Q/col question representation weighted by column attentions in SQLNet paper
        q_weighted = (q_enc.unsqueeze(1) * att_prob_qc.unsqueeze(3)).sum(2)
        # Compute prediction scores
        # self.col_out.squeeze(): (B, max_col_len)
        # col_score computed by P_where(col/Q) of equation 2 without softmax/sigmoid in SQLNet paper
        col_score = self.col_out(self.col_out_q(q_weighted) + self.col_out_c(col_enc)).squeeze()
        for idx, num in enumerate(col_len):
            if num < max_col_len:
                col_score[idx, num:] = -100
                
        # get select columns for op prediction because op depends on column
        chosen_col_gt = []
        if gt_cond is None: # if no gold cond column provided, use predicted cond
            cond_nums = np.argmax(col_num_score.data.cpu().numpy(), axis=1)
            col_scores = col_score.data.cpu().numpy()
            chosen_col_gt = [list(np.argsort(-col_scores[b])[:cond_nums[b]]) for b in range(len(cond_nums))]
        else:
            chosen_col_gt = [[x[0] for x in one_gt_cond] for one_gt_cond in gt_cond]
        # get embeddings for gold or predicted cond columns
        col_emb = []
        for b in range(B):
            cur_col_emb = torch.stack([col_enc[b, x]
                for x in chosen_col_gt[b]] + [col_enc[b, 0]] * (5 - len(chosen_col_gt[b])))
            col_emb.append(cur_col_emb)
        col_emb = torch.stack(col_emb)

        ## Predict operators based on gold/predicted condition columns ## 
        op_att_val = torch.matmul(self.op_att(q_enc).unsqueeze(1),
                col_emb.unsqueeze(3)).squeeze()
        for idx, num in enumerate(q_len):
            if num < max_q_len:
                op_att_val[idx, :, num:] = -100
        op_att = self.softmax(op_att_val.view(-1, max_q_len)).view(B, -1, max_q_len)
        # E_Q/pred_or_gold_cols
        q_weighted_op = (q_enc.unsqueeze(1) * op_att.unsqueeze(3)).sum(2)
        # P_op(i/Q,col) in the paper
        op_score = self.op_out(self.op_out_q(q_weighted_op) +
                            self.op_out_c(col_emb)).squeeze()

        score = (col_num_score, col_score, op_score)

        return score

**TODO**: Please follow comments in the `WHERE` condition module to understand all steps listed above. You need to implement the `ORDERBY` module below. Consider `ORDER BY SUM(population) DESC LIMIT 1`.

In [0]:
#TODO!
class OrderPredictor(nn.Module):
    '''
    module to predict orderby column number, columns, and aggregations in ORDER clause.
    '''
    def __init__(self, N_word, N_h, N_depth):
        super(OrderPredictor, self).__init__()
        self.N_h = N_h
       
        #TODO:IMPLEMENT YOUR CODE BELOW
        #initialize LSTM for encoding questions
        self.q_lstm = nn.LSTM(input_size=N_word, hidden_size=N_h//2,
                num_layers=N_depth, batch_first=True,
                dropout=0.3, bidirectional=True)
        
        #initialize LSTM for encoding column names
        self.col_lstm = nn.LSTM(input_size=N_word, hidden_size=N_h//2,
                num_layers=N_depth, batch_first=True,
                dropout=0.3, bidirectional=True)
        
        #initialize functions for orderby column number prediction
        #only consider 0 or 1
        self.q_num_att = nn.Linear(N_h, N_h)
        self.col_num_out_q = nn.Linear(N_h, N_h)
        self.col_num_out = nn.Sequential(nn.Tanh(), nn.Linear(N_h, 2))
        
        #initialize functions for orderby column prediction
        self.q_att = nn.Linear(N_h, N_h)
        self.col_out_q = nn.Linear(N_h, N_h)
        self.col_out_c = nn.Linear(N_h, N_h)
        self.col_out = nn.Sequential(nn.Tanh(), nn.Linear(N_h, 1))
        
        #initialize functions for orderby aggregation prediction
        #here we assume there is at most one aggregator
        #remember: agg ouput is probs over ('none', 'max', 'min', 'count', 'sum', 'avg')
        self.agg_att = nn.Linear(N_h, N_h)
        self.agg_out_q = nn.Linear(N_h, N_h)
        self.agg_out_c = nn.Linear(N_h, N_h)
        self.agg_out = nn.Sequential(nn.Tanh(), nn.Linear(N_h, 6)) 
        
        #initialize functions for orderby parity
        #possible values are (none, desc, asc, desc limit, asc limit)
        self.par_att = nn.Linear(N_h, N_h)
        self.par_out_q = nn.Linear(N_h, N_h)
        self.par_out_c = nn.Linear(N_h, N_h)
        self.par_out = nn.Sequential(nn.Tanh(), nn.Linear(N_h, 5)) 
        
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q_emb_var, q_len, col_emb_var, col_len, col_name_len):
        '''
        Forward pass for orderby prodection. 
        Note: we can use different ways to compute output probs for each module. 
        For simplicity, please follow exact the same ways as other modules do.
        
        ---Parameters---
        
        please refer to CondPredictor
        '''
        max_q_len = max(q_len)
        max_col_len = max(col_len)
        B = len(q_len)
        
        #forward pass through LSTM for questions and column names
        q_enc, _ = run_lstm(self.q_lstm, q_emb_var, q_len)
        col_enc = col_name_encode(col_emb_var, col_name_len, col_len, self.col_lstm)
        
        #TODO:IMPLEMENT YOUR CODE BELOW
        ## Predict orderby column number 0 or 1 ##
        att_val_qc_num = torch.bmm(col_enc, self.q_num_att(q_enc).transpose(1, 2))
        for idx, num in enumerate(col_len):
            if num < max_col_len:
                att_val_qc_num[idx, num:, :] = -100
        for idx, num in enumerate(q_len):
            if num < max_q_len:
                att_val_qc_num[idx, :, num:] = -100
        att_prob_qc_num = self.softmax(att_val_qc_num.view((-1, max_q_len))).view(B, -1, max_q_len)
        q_weighted_num = (q_enc.unsqueeze(1) * att_prob_qc_num.unsqueeze(3)).sum(2).sum(1)
        col_num_score = self.col_num_out(self.col_num_out_q(q_weighted_num))
        
        ## Predict orderby column ##
        att_val_qc = torch.bmm(col_enc, self.q_att(q_enc).transpose(1, 2))
        for idx, num in enumerate(q_len):
            if num < max_q_len:
                att_val_qc[idx, :, num:] = -100
        att_prob_qc = self.softmax(att_val_qc.view((-1, max_q_len))).view(B, -1, max_q_len)
        q_weighted = (q_enc.unsqueeze(1) * att_prob_qc.unsqueeze(3)).sum(2)
        col_score = self.col_out(self.col_out_q(q_weighted) + self.col_out_c(col_enc)).squeeze()
        for idx, num in enumerate(col_len):
            if num < max_col_len:
                col_score[idx, num:] = -100
        
        ## Predict orderby aggregation ##
        agg_att_val = torch.bmm(col_enc, self.agg_att(q_enc).transpose(1, 2))
        for idx, num in enumerate(col_len):
            if num < max_col_len:
                agg_att_val[idx, num:, :] = -100
        for idx, num in enumerate(q_len):
            if num < max_q_len:
                agg_att_val[idx, :, num:] = -100
        agg_att = self.softmax(agg_att_val.view((-1, max_q_len))).view(B, -1, max_q_len)
        q_weighted_agg = (q_enc.unsqueeze(1) * agg_att.unsqueeze(3)).sum(2).sum(1)
        agg_score = self.agg_out(self.agg_out_q(q_weighted_agg))
        
        ## Predict if none, desc, asc , desc limit, asc limit ##
        par_att_val = torch.bmm(col_enc, self.par_att(q_enc).transpose(1, 2))
        for idx, num in enumerate(col_len):
            if num < max_col_len:
                par_att_val[idx, num:, :] = -100
        for idx, num in enumerate(q_len):
            if num < max_q_len:
                par_att_val[idx, :, num:] = -100
        par_att = self.softmax(par_att_val.view((-1, max_q_len))).view(B, -1, max_q_len)
        q_weighted_par = (q_enc.unsqueeze(1) * par_att.unsqueeze(3)).sum(2).sum(1)
        par_score = self.par_out(self.par_out_q(q_weighted_par))

        score = (col_num_score, col_score, agg_score, par_score)

        return score

#### 5. SQLNet Model

SQLNet main module calls all submodules to predict all components in SQL queries. It also computes the loss and accuracy of all submodules. For simplicity, we do not include functions that convert predictions into real SQL queries.


**TODO**: Again,  the code is long, so you can just **SCAN** all code related to the `SELECT` and `GROUPBY` modules. Please follow comments for the code related to the `WHERE` module to understand what is going on there. You need to compute the loss and accuracy for the `ORDERBY` module.

In [0]:
#READCODE! But just SCAN all code related to SELECT and GROUPBY modules.
#TODO!
class SQLNet(nn.Module):
    '''
    SQLNet main module that calls all sub-modules
    '''
    def __init__(self, word_emb, N_word, N_h=120, N_depth=2):
        super(SQLNet, self).__init__()
        self.N_h = N_h
        self.N_depth = N_depth

        self.max_col_num = 45
        self.max_tok_num = 200
        self.COND_OPS = ['EQL', 'GT', 'LT']

        self.embed_layer = WordEmbedding(word_emb, N_word)

        #select predictor initialization
        self.sel_pred = SelPredictor(N_word, N_h, N_depth)
        #where condition predictor initialization
        self.cond_pred = CondPredictor(N_word, N_h, N_depth)
        #groupby predictor initialization
        self.group_pred = GroupPredictor(N_word, N_h, N_depth)
        #orderby predictor initialization
        self.order_pred = OrderPredictor(N_word, N_h, N_depth)
        
        #loss function
        self.CE = nn.CrossEntropyLoss()
        self.softmax = nn.Softmax(dim=-1)
        self.sigm = nn.Sigmoid()

    def forward(self, q, col, gt_cond=None, gt_sel=None):
        '''
        forward pass for all submodules in SQLNet model
        
        ---Parameters---
        
        q (list of lists): each list include words in each question
        cols (list of lists of lists): list (batch of databases) of lists 
              (column names in each database) of lists (words in each column)
        gt_cond(list of lists): list of gold columns in condition, it is None during testing 
        gt_sel (list of lists): list of gold columns in select, it is None during testing
        
        ---Returns---
        
        scores (tuple): prediction scores for each submodule
        
        '''
        B = len(q)

        sel_score = None
        cond_score = None
        group_score = None
        order_score = None
        #map words into embeddings
        x_emb_var, x_len = self.embed_layer.gen_x_batch(q)
        col_inp_var, col_name_len, col_len = self.embed_layer.gen_col_batch(col)
        max_x_len = max(x_len)
        #run forward pass for each submodule
        sel_score = self.sel_pred(x_emb_var, x_len, col_inp_var, col_len, col_name_len, gt_sel=gt_sel)
        cond_score = self.cond_pred(x_emb_var, x_len, col_inp_var, col_len, col_name_len, gt_cond=gt_cond)
        group_score = self.group_pred(x_emb_var, x_len, col_inp_var, col_len, col_name_len)
        order_score = self.order_pred(x_emb_var, x_len, col_inp_var, col_len, col_name_len)

        scores = (sel_score, cond_score, group_score, order_score)
        
        return scores


    def loss(self, score, truth_num):
        '''
        compute loss for each submodule prediction
        
        ---Parameters---
        
        score (tuple): scores returned by the forward pass
        truth_num (list of tuples): each tuple in the list contains detailed gold
              labels. refer to ans_seq in to_batch_seq function
        
        ---Returns---
        
        loss (scalar tensor): loss of all submodule
        '''
        sel_score, cond_score, group_score, order_score = score

        sel_num_score, sel_col_score, agg_num_score, agg_op_score = sel_score
        cond_num_score, cond_col_score, cond_op_score = cond_score
        gby_num_score, gby_score, hv_score, hv_col_score, hv_agg_score, hv_op_score = group_score
        ody_num_score, ody_col_score, ody_agg_score, ody_par_score = order_score

        B = len(truth_num)
        loss = 0
        
        #----------loss for sel_pred -------------#

        # loss for sel agg # and sel agg
        for b in range(len(truth_num)):
            curr_col = truth_num[b][1][0]
            curr_col_num_aggs = 0
            gt_aggs_num = []
            for i, col in enumerate(truth_num[b][1]):
                if col != curr_col:
                    gt_aggs_num.append(curr_col_num_aggs)
                    curr_col = col
                    curr_col_num_aggs = 0
                if truth_num[b][0][i] != 0:
                    curr_col_num_aggs += 1
            gt_aggs_num.append(curr_col_num_aggs)
            agg_num_truth_var = torch.from_numpy(np.array(gt_aggs_num)).to(device) #supposed to be gt # of aggs
            agg_num_pred = agg_num_score[b, :truth_num[b][5]] # supposed to be gt # of select columns
            loss += (self.CE(agg_num_pred, agg_num_truth_var) \
                    / len(truth_num))
            # loss for sel agg prediction
            T = 6 #num agg ops
            truth_prob = np.zeros((truth_num[b][5], T), dtype=np.float32)
            gt_agg_by_sel = []
            curr_sel_aggs = []
            curr_col = truth_num[b][1][0]
            col_counter = 0
            for i, col in enumerate(truth_num[b][1]):
                if col != curr_col:
                    gt_agg_by_sel.append(curr_sel_aggs)
                    curr_col = col
                    col_counter += 1
                    curr_sel_aggs = [truth_num[b][0][i]]
                    truth_prob[col_counter][curr_sel_aggs] = 1
                else:
                    curr_sel_aggs.append(truth_num[b][0][i])
                    truth_prob[col_counter][curr_sel_aggs] = 1
            agg_op_truth_var = torch.from_numpy(truth_prob).to(device)
            agg_op_prob = self.sigm(agg_op_score[b, :truth_num[b][5]])
            agg_bce_loss = -torch.mean( 3*(agg_op_truth_var * \
                    torch.log(agg_op_prob+1e-10)) + \
                    (1-agg_op_truth_var) * torch.log(1-agg_op_prob+1e-10) )
            loss += agg_bce_loss / len(truth_num)

        #Evaluate the number of select columns
        sel_num_truth = list(map(lambda x: x[5]-1, truth_num)) #might need to be the length of the set of columms
        sel_num_truth_var = torch.from_numpy(np.array(sel_num_truth)).to(device)
        loss += self.CE(sel_num_score, sel_num_truth_var)
        # Evaluate the select columns
        T = len(sel_col_score[0])
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            truth_prob[b][truth_num[b][1]] = 1
        sel_col_truth_var = torch.from_numpy(truth_prob).to(device)
        sel_col_prob = self.sigm(sel_col_score)
         #weighted negative log-likelihood loss(col, Q, y) in SQLNet paper
        sel_bce_loss = -torch.mean( 3*(sel_col_truth_var * \
                torch.log(sel_col_prob+1e-10)) + \
                (1-sel_col_truth_var) * torch.log(1-sel_col_prob+1e-10) )
        loss += sel_bce_loss
        
        #----------------loss for cond_pred--------------------#
        
        #Evaluate the number of conditions, cross entropy loss used
        cond_num_truth = list(map(lambda x:x[2], truth_num))
        cond_num_truth_var = torch.from_numpy(np.array(cond_num_truth)).to(device)
        loss += self.CE(cond_num_score, cond_num_truth_var)
        #Evaluate the columns of conditions
        T = len(cond_col_score[0])
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            if len(truth_num[b][3]) > 0:
                truth_prob[b][list(truth_num[b][3])] = 1

        cond_col_truth_var = torch.from_numpy(truth_prob).to(device)
        cond_col_prob = self.sigm(cond_col_score)
        #weighted negative log-likelihood bce loss(col, Q, y) in SQLNet paper
        bce_loss = -torch.mean( 3*(cond_col_truth_var * \
                torch.log(cond_col_prob+1e-10)) + \
                (1-cond_col_truth_var) * torch.log(1-cond_col_prob+1e-10) )
        loss += bce_loss
        #Evaluate the operator of conditions
        for b in range(len(truth_num)):
            if len(truth_num[b][4]) == 0:
                continue
            cond_op_truth_var = torch.from_numpy(np.array(truth_num[b][4])).to(device)
            cond_op_pred = cond_op_score[b, :len(truth_num[b][4])]
            loss += (self.CE(cond_op_pred, cond_op_truth_var) \
                    / len(truth_num))
          
        # -----------loss for group_pred -------------- #
        
        # Evaluate the number of group by columns
        gby_num_truth = list(map(lambda x: x[7], truth_num))
        gby_num_truth_var = torch.from_numpy(np.array(gby_num_truth)).to(device)
        loss += self.CE(gby_num_score, gby_num_truth_var)
        # Evaluate the group by columns
        T = len(gby_score[0])
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            if len(truth_num[b][6]) > 0:
                truth_prob[b][list(truth_num[b][6])] = 1
        gby_col_truth_var = torch.from_numpy(truth_prob).to(device)
        gby_col_prob = self.sigm(gby_score)
        gby_bce_loss = -torch.mean( 3*(gby_col_truth_var * \
                torch.log(gby_col_prob+1e-10)) + \
                (1-gby_col_truth_var) * torch.log(1-gby_col_prob+1e-10) )
        loss += gby_bce_loss
        # Evaluate having
        having_truth = [1 if len(x[13]) == 1 else 0 for x in truth_num]
        having_truth_var = torch.from_numpy(np.array(having_truth)).to(device)
        loss += self.CE(hv_score, having_truth_var)
        # Evaluate having col
        T = len(hv_col_score[0])
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            if len(truth_num[b][13]) > 0:
                truth_prob[b][truth_num[b][13]] = 1
        hv_col_truth_var = torch.from_numpy(truth_prob).to(device)
        hv_col_prob = self.sigm(hv_col_score)
        hv_col_bce_loss = -torch.mean( 3*(hv_col_truth_var * \
                torch.log(hv_col_prob+1e-10)) + \
                (1-hv_col_truth_var) * torch.log(1-hv_col_prob+1e-10) )
        loss += hv_col_bce_loss
        # Evaluate having agg
        T = len(hv_agg_score[0])
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            if len(truth_num[b][12]) > 0:
                truth_prob[b][truth_num[b][12]] = 1
        hv_agg_truth_var = torch.from_numpy(truth_prob).to(device)
        hv_agg_prob = self.sigm(hv_agg_truth_var)
        hv_agg_bce_loss = -torch.mean( 3*(hv_agg_truth_var * \
                torch.log(hv_agg_prob+1e-10)) + \
                (1-hv_agg_truth_var) * torch.log(1-hv_agg_prob+1e-10) )
        loss += hv_agg_bce_loss
        # Evaluate having op
        T = len(hv_op_score[0])
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            if len(truth_num[b][14]) > 0:
                truth_prob[b][truth_num[b][14]] = 1
        hv_op_truth_var = torch.from_numpy(truth_prob).to(device)
        hv_op_prob = self.sigm(hv_op_truth_var)
        hv_op_bce_loss = -torch.mean( 3*(hv_op_truth_var * \
                torch.log(hv_op_prob+1e-10)) + \
                (1-hv_op_truth_var) * torch.log(1-hv_op_prob+1e-10) )
        loss += hv_op_bce_loss

        
        #TODO:IMPLEMENT YOUR CODE BELOW
        # -----------loss for order_pred -------------- #
        
        # Evaluate the number of order by columns, use cross entropy loss
        ody_num_truth = list(map(lambda x: x[10], truth_num))
        ody_num_truth_var = torch.from_numpy(np.array(ody_num_truth)).to(device)
        loss += self.CE(ody_num_score, ody_num_truth_var)
        
        # Evaluate the order by columns, use weighted negative log-likelihood bce loss
        T = len(ody_col_score[0])
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            if len(truth_num[b][9]) > 0:
                truth_prob[b][list(truth_num[b][9])] = 1
        ody_col_truth_var = torch.from_numpy(truth_prob).to(device)
        ody_col_prob = self.sigm(ody_col_score)
        ody_col_bce_loss = -torch.mean( 3*(ody_col_truth_var * \
                torch.log(ody_col_prob+1e-10)) + \
                (1-ody_col_truth_var) * torch.log(1-ody_col_prob+1e-10) )
        loss += ody_col_bce_loss
        
        # Evaluate order agg assume only one, use weighted negative log-likelihood bce loss
        T = len(ody_agg_score[0])   # 6
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            if len(truth_num[b][9]) > 0:
                truth_prob[b][list(truth_num[b][8])] = 1
        ody_agg_truth_var = torch.from_numpy(truth_prob).to(device)
        ody_agg_prob = self.sigm(ody_agg_score)
        ody_agg_bce_loss = -torch.mean( 3*(ody_agg_truth_var * \
                torch.log(ody_agg_prob+1e-10)) + \
                (1-ody_agg_truth_var) * torch.log(1-ody_agg_prob+1e-10) )
        loss += ody_agg_bce_loss 

        # Evaluate orderby parity, use cross entropy loss
        ody_par_truth = list(map(lambda x: x[11], truth_num))
        ody_par_truth_var = torch.from_numpy(np.array(ody_par_truth)).to(device)
        loss += self.CE(ody_par_score, ody_par_truth_var)        
        
        return loss

      
    def check_acc(self, pred_queries, gt_queries):
        '''
        report accuracy of predictions for each submodule
        
        ---Parameters---
        
        pred_queries (list of dicts): each dict is the prediction result converted
               by gen_query function based on predicted scores of each submodule
        gt_queries (list of dicts): each dict the gold prediction result for each
               data point
        
        ---Returns---
        
        err: error numbers for sel, cond, groupby, orderby, and all
        
        '''
        B = len(gt_queries)

        tot_err = 0.0
        sel_err = agg_num_err = agg_op_err = sel_num_err = sel_col_err = 0.0
        cond_err = cond_num_err = cond_col_err = cond_op_err = 0.0
        gby_err = gby_num_err = gby_col_err = hv_err = hv_col_err = hv_agg_err = hv_op_err = 0.0
        ody_err = ody_num_err = ody_col_err = ody_agg_err = ody_par_err = 0.0
        
        for b, (pred_qry, gt_qry) in enumerate(zip(pred_queries, gt_queries)):

            good = True
            tot_flag = True
            sel_flag = True
            cond_flag = True
            gby_flag = True
            ody_flag = True
            
            # accuracy for sel
            sel_gt = gt_qry['sel']
            sel_num_gt = len(set(sel_gt))
            sel_pred = pred_qry['sel']
            sel_num_pred = pred_qry['sel_num']
            if sel_num_pred != sel_num_gt:
                sel_num_err += 1
                sel_flag = False
            if sorted(set(sel_pred)) != sorted(set(sel_gt)):
                sel_col_err += 1
                sel_flag = False

            agg_gt = gt_qry['agg']
            curr_col = gt_qry['sel'][0]
            curr_col_num_aggs = 0
            gt_aggs_num = []
            gt_sel_order = [curr_col]
            for i, col in enumerate(gt_qry['sel']):
                if col != curr_col:
                    gt_sel_order.append(col)
                    gt_aggs_num.append(curr_col_num_aggs)
                    curr_col = col
                    curr_col_num_aggs = 0
                if agg_gt[i] != 0:
                    curr_col_num_aggs += 1
            gt_aggs_num.append(curr_col_num_aggs)

            if pred_qry['agg_num'] != gt_aggs_num:
                agg_num_err += 1
                sel_flag = False

            if sorted(pred_qry['agg']) != sorted(gt_qry['agg']): # naive
                agg_op_err += 1
                sel_flag = False

            if not sel_flag:
                sel_err += 1
                good = False
                
            # accuracy for conds
            cond_pred = pred_qry['conds']
            cond_gt = gt_qry['cond']
            flag = True
            if len(cond_pred) != len(cond_gt):
                flag = False
                cond_num_err += 1
                cond_flag = False
            if flag and set(x[0] for x in cond_pred) != set(x[0] for x in cond_gt):
                flag = False
                cond_col_err += 1
                cond_flag = False
            for idx in range(len(cond_pred)):
                if not flag:
                    break
                gt_idx = tuple(x[0] for x in cond_gt).index(cond_pred[idx][0])
                if flag and cond_gt[gt_idx][1] != cond_pred[idx][1]:
                    flag = False
                    cond_op_err += 1
                    cond_flag = False

            if not cond_flag:
                cond_err += 1
                good = False
                
            # accuracy for group
            gby_gt = gt_qry['group'][:-1]
            gby_pred = pred_qry['group']
            gby_num_pred = pred_qry['gby_num']
            gby_num_gt = len(gby_gt)
            if gby_num_pred != gby_num_gt:
                gby_num_err += 1
                gby_flag = False
            if sorted(gby_pred) != sorted(gby_gt):
                gby_col_err += 1
                gby_flag = False
            gt_gby_agg = gt_qry['group'][-1][0]
            gt_gby_col = gt_qry['group'][-1][1]
            gt_gby_op = gt_qry['group'][-1][2]
            if gby_num_pred != 0 and len(gt_gby_col) != 0:
                if pred_qry['hv'] != 1:
                    hv_err += 1
                    gby_flag = False
                if pred_qry['hv_agg'] != gt_gby_agg[0]:
                    hv_agg_err += 1
                    gby_flag = False
                if pred_qry['hv_col'] != gt_gby_col[0]:
                    hv_col_err += 1
                    gby_flag = False
                if pred_qry['hv_op'] != gt_gby_op[0]:
                    hv_op_err += 1
                    gby_flag = False

            if not gby_flag:
                gby_err += 1
                good = False

            # accuracy for order
            ody_gt_aggs = gt_qry['order'][0]
            ody_gt_cols = gt_qry['order'][1]
            ody_gt_par = gt_qry['order'][2]
            ody_num_cols_pred = pred_qry['ody_num']
            ody_cols_pred = pred_qry['order']
            ody_aggs_pred = pred_qry['ody_agg']
            ody_par_pred = pred_qry['parity']
            
            #TODO:IMPLEMENT YOUR CODE BELOW
            # check if orderby column number prediction is right
            if ody_num_cols_pred != len(ody_gt_cols):
                ody_num_err += 1
                ody_flag = False
            
            # if gold orderby col number is not 0, compute accuracy for 
            # 1. orderby column, 2. orderby agg, 3. orderby parity
            if len(ody_gt_cols) > 0:
                if ody_cols_pred != ody_gt_cols:
                    ody_col_err += 1
                    ody_flag = False
                if ody_aggs_pred != ody_gt_aggs:
                    ody_agg_err += 1
                    ody_flag = False
                if ody_par_pred != ody_gt_par:
                    ody_par_err += 1
                    ody_flag = False
            
            # update ody_err and 'good' flag
            if not ody_flag:
                ody_err += 1
                good = False
            
            if not good:
                tot_err += 1

        return np.array((sel_err, cond_err, gby_err, ody_err)), tot_err


    def gen_query(self, score, q, col, raw_q, raw_col, verbose=False):
        '''
        generate query prediction according to predicted scores of each submodule
        '''
        sel_score, cond_score, group_score, order_score = score

        sel_num_score, sel_col_score, agg_num_score, agg_op_score = [x.data.cpu().numpy() if x is not None else None for x in sel_score]
        cond_num_score, cond_col_score, cond_op_score = [x.data.cpu().numpy() if x is not None else None for x in cond_score]
        gby_num_score, gby_score, hv_score, hv_col_score, hv_agg_score, hv_op_score = [x.data.cpu().numpy() if x is not None else None for x in group_score]
        ody_num_score, ody_col_score, ody_agg_score, ody_par_score = [x.data.cpu().numpy() if x is not None else None for x in order_score]
        
        ret_queries = []
        B = len(cond_num_score)
        for b in range(B):
            cur_query = {}
             # ------------get sel predict
            sel_num_cols = np.argmax(sel_num_score[b]) + 1
            cur_query['sel_num'] = sel_num_cols
            cur_query['sel'] = np.argsort(-sel_col_score[b])[:sel_num_cols]

            agg_nums = []
            agg_preds = []
            for idx in range(sel_num_cols):
                curr_num_aggs = np.argmax(agg_num_score[b][idx])
                agg_nums.append(curr_num_aggs)
                if curr_num_aggs == 0:
                    curr_agg_ops = [0]
                else:
                    curr_agg_ops = [x for x in list(np.argsort(-agg_op_score[b][idx])) if x != 0][:curr_num_aggs]
                agg_preds += curr_agg_ops
            cur_query['agg_num'] = agg_nums
            cur_query['agg'] = agg_preds
            
            #---------get cond predict
            cur_query['conds'] = []
            cond_num = np.argmax(cond_num_score[b])
            max_idxes = np.argsort(-cond_col_score[b])[:cond_num]
            for idx in range(cond_num):
                cur_cond = []
                cur_cond.append(max_idxes[idx])
                cur_cond.append(np.argmax(cond_op_score[b][idx]))
                cur_query['conds'].append(cur_cond)
            
            #----------get group by predict
            gby_num_cols = np.argmax(gby_num_score[b])
            cur_query['gby_num'] = gby_num_cols
            cur_query['group'] = np.argsort(-gby_score[b])[:gby_num_cols]
            cur_query['hv'] = np.argmax(hv_score[b])
            if gby_num_cols != 0 and cur_query['hv'] != 0:
                cur_query['hv_agg'] = np.argmax(hv_agg_score[b])
                cur_query['hv_col'] = np.argmax(hv_col_score[b])
                cur_query['hv_op'] = np.argmax(hv_op_score[b])
            else:
                cur_query['hv'] = 0
                cur_query['hv_agg'] = 0
                cur_query['hv_col'] = -1
                cur_query['hv_op'] = -1
                
            # --------get order by
            ody_num_cols = np.argmax(ody_num_score[b])
            cur_query['ody_num'] = ody_num_cols
            cur_query['order'] = np.argsort(-ody_col_score[b])[:ody_num_cols]
            if ody_num_cols != 0:
                cur_query['ody_agg'] = np.argmax(ody_agg_score[b])
                cur_query['parity'] = np.argmax(ody_par_score[b])
            else:
                cur_query['ody_agg'] = 0
                cur_query['parity'] = -1
                
            ret_queries.append(cur_query)

        return ret_queries

#### 6. Batching

Generate batches of input samples.

In [0]:
#READCODE! understand final inputs to the model
def to_batch_seq(sql_data, idxes, start, end, ret_vis_data=False):
    '''
    get batch inputs for the model
    
    ---Parameters---
    
    sql_data (list of dicts): each dict in the list is a training data point 
            with question, database info, and target labels for each module 
            (agg/sel/SELECT, cond/WHERE, group/GROUP, order/ORDER) in SQLNet
    idxes, start, end: indices for batching
    
    ---Returns---
    too many... please check them by yourself
    '''
    q_seq = []
    col_seq = []
    col_num = []
    ans_seq = []
    query_seq = []
    gt_cond_seq = []
    vis_seq = []

    col_org_seq = []

    for i in range(start, end):
        sql = sql_data[idxes[i]]
        col_org_seq.append(sql['col_org'])
        q_seq.append(sql['question_tok'])
        column_names = sql["column_names"]
        col_num.append(len(column_names))
        tab_cols = [col[1] for col in column_names]
        col_seq.append([x.split(" ") for x in tab_cols])
        ans_seq.append((sql['agg'],     # sel agg # 0
            sql['sel'],                 # sel col # 1
            len(sql['cond']),           # cond # 2
            tuple(x[0] for x in sql['cond']), # cond col 3
            tuple(x[1] for x in sql['cond']), # cond op 4
            len(set(sql['sel'])),       # number of unique select cols 5
            sql['group'][:-1],          # group by columns 6
            len(sql['group']) - 1,      # number of group by columns 7
            sql['order'][0],            # order by aggregations 8
            sql['order'][1],            # order by columns 9
            len(sql['order'][1]),       # num order by columns 10
            sql['order'][2],            # order by parity 11
            sql['group'][-1][0],        # having agg 12
            sql['group'][-1][1],        # having col 13
            sql['group'][-1][2]         # having op 14
            ))
        #order: [[agg], [col], [dat]]
        #group: [col1, col2, [agg, col, op]]
        query_seq.append(sql['query_tok'])
        gt_cond_seq.append([x for x in sql['cond']])
        vis_seq.append((sql['question'], tab_cols, sql['query']))

    if ret_vis_data:
        return q_seq, col_seq, col_num, ans_seq, query_seq, gt_cond_seq, vis_seq, col_org_seq
    else:
        return q_seq, col_seq, col_num, ans_seq, query_seq, gt_cond_seq, col_org_seq

#### 7. Model Traing and Testing

Initialize SQLNet and optimizer below:

In [0]:
N_word=50 #set embedding dimension to 50
#initialize SQLNet model, and move it to device (cuda or cpu)
model = SQLNet(word_emb, N_word=N_word).to(device)
learning_rate = 1e-3
#initialize optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0)

Run the train and test procedures on one epoch of the data:

In [0]:
#READCODE!
def epoch_train(model, optimizer, batch_size, sql_data):
    '''
    train model on one epoch of the data
    '''
    model.train() #set to training mode
    perm=np.random.permutation(len(sql_data))
    cum_loss = 0.0
    st = 0
    while st < len(sql_data): #until one epoch
        ed = st+batch_size if st+batch_size < len(perm) else len(perm)
        #batching data input
        q_seq, col_seq, col_num, ans_seq, query_seq, gt_cond_seq, col_org_seq = \
                to_batch_seq(sql_data, perm, st, ed)
        gt_sel_seq = [x[1] for x in ans_seq]
        #1. call forward pass of SQLNet, 2.compute loss
        score = model.forward(q_seq, col_seq, gt_cond=gt_cond_seq, gt_sel=gt_sel_seq)
        #compute loss
        loss = model.loss(score, ans_seq)
        #add loss
        cum_loss += loss.item()*(ed - st)
        #recognize them?
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        st = ed

    return cum_loss / len(sql_data)


def epoch_acc(model, batch_size, sql_data):
    '''
    check accuracy of the model
    '''
    model.eval() # set to evaluation mode
    perm = list(range(len(sql_data)))
    st = 0
    one_acc_num = 0.0
    tot_acc_num = 0.0
    while st < len(sql_data): #until one epoch
        ed = st+batch_size if st+batch_size < len(perm) else len(perm)
        #batching data input
        q_seq, col_seq, col_num, ans_seq, query_seq, gt_cond_seq,\
         raw_data, col_org_seq = to_batch_seq(sql_data, perm, st, ed, ret_vis_data=True)
        raw_q_seq = [x[0] for x in raw_data]
        raw_col_seq = [x[1] for x in raw_data]
        #batching gold query
        query_gt, table_ids = to_batch_query(sql_data, perm, st, ed)
        #forward pass
        score = model.forward(q_seq, col_seq)
        #generate predicted query
        pred_queries = model.gen_query(score, q_seq, col_seq, raw_q_seq, raw_col_seq)
        #check accuracy
        one_err, tot_err = model.check_acc(pred_queries, query_gt)
        #add error number
        one_acc_num += (ed-st-one_err)
        tot_acc_num += (ed-st-tot_err)

        st = ed
    return tot_acc_num / len(sql_data), one_acc_num / len(sql_data)

Finally, we can run the train and test processes, print the results, and save the models:

**Note**: We already include enough info in this assignment. So the final evaluation step that computes the set matching accuracy of complete predicted and gold SQL queries is not covered here. You can refer to [the Spider Github page](https://github.com/taoyds/spider/tree/master/evaluation_examples) for more details if you are interested.

In [19]:
#READCODE!
BATCH_SIZE=64
#initial accuracy
init_acc = epoch_acc(model, BATCH_SIZE, val_sql_data)
best_sel_acc = init_acc[1][0]
best_cond_acc = init_acc[1][1]
best_group_acc = init_acc[1][2]
best_order_acc = init_acc[1][3]
best_tot_acc = 0.0

for i in range(30):
    print('Epoch %d @ %s'%(i+1, datetime.datetime.now()))
    print(' Loss = %s'%epoch_train(model, optimizer, BATCH_SIZE, train_sql_data))
    train_tot_acc, train_bkd_acc = epoch_acc(model, BATCH_SIZE, train_sql_data)
    print(' Train acc_qm: %s' % train_tot_acc)
    print(' Breakdown results: sel: %s, cond: %s, group: %s, order: %s'\
        % (train_bkd_acc[0], train_bkd_acc[1], train_bkd_acc[2], train_bkd_acc[3]))

    val_tot_acc, val_bkd_acc = epoch_acc(model, BATCH_SIZE, val_sql_data) #for detailed error analysis, pass True to error_print
    print(' Dev acc_qm: %s' % val_tot_acc)
    print(' Breakdown results: sel: %s, cond: %s, group: %s, order: %s'\
        % (val_bkd_acc[0], val_bkd_acc[1], val_bkd_acc[2], val_bkd_acc[3]))

    #save models
    if val_bkd_acc[0] > best_sel_acc:
        best_sel_acc = val_bkd_acc[0]
        print("Saving sel model...")
        torch.save(model.sel_pred.state_dict(), os.path.join(SAVED_MODEL_DIR, "sel_models.dump"))
    if val_bkd_acc[1] > best_cond_acc:
        best_cond_acc = val_bkd_acc[1]
        print("Saving cond model...")
        torch.save(model.cond_pred.state_dict(), os.path.join(SAVED_MODEL_DIR, "cond_models.dump"))
    if val_bkd_acc[2] > best_group_acc:
        best_group_acc = val_bkd_acc[2]
        print("Saving group model...") 
        torch.save(model.group_pred.state_dict(), os.path.join(SAVED_MODEL_DIR, "group_models.dump"))
    if val_bkd_acc[3] > best_order_acc:
        best_order_acc = val_bkd_acc[3]
        print("Saving order model...")
        torch.save(model.order_pred.state_dict(), os.path.join(SAVED_MODEL_DIR, "order_models.dump"))
    if val_tot_acc > best_tot_acc:
        best_tot_acc = val_tot_acc


    print(' Best val sel = %s, cond = %s, group = %s, order = %s, tot = %s'%(best_sel_acc, best_cond_acc, best_group_acc, best_order_acc, best_tot_acc))

Epoch 1 @ 2019-04-09 21:49:55.605207
 Loss = 7.2532005255562915


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:410: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


 Train acc_qm: 0.011285714285714286
 Breakdown results: sel: 0.08714285714285715, cond: 0.36842857142857144, group: 0.7508571428571429, order: 0.7304285714285714
 Dev acc_qm: 0.008704061895551257
 Breakdown results: sel: 0.10638297872340426, cond: 0.3413926499032882, group: 0.7350096711798839, order: 0.7079303675048356
Saving sel model...
Saving cond model...
Saving group model...
Saving order model...
 Best val sel = 0.10638297872340426, cond = 0.3413926499032882, group = 0.7350096711798839, order = 0.7079303675048356, tot = 0.008704061895551257
Epoch 2 @ 2019-04-09 21:52:20.139362
 Loss = 5.317157589503697
 Train acc_qm: 0.008285714285714285
 Breakdown results: sel: 0.09471428571428571, cond: 0.43257142857142855, group: 0.6728571428571428, order: 0.7961428571428572
 Dev acc_qm: 0.005802707930367505
 Breakdown results: sel: 0.11798839458413926, cond: 0.42940038684719534, group: 0.648936170212766, order: 0.7775628626692457
Saving sel model...
Saving cond model...
Saving order model...


**TODO**: Now you implemented one module in SQLNet. List two limitations of SQLNet model and two possible ideas for the Spider task. - ___10 points___ <br/>
**YOUR ANSWER:** 

Limitations of SQLNet:
1. SQLNet doesn't handle scenarios where nested SQL queries are needed. 
2. The ORDER BY module considers only 0 or 1 column, but in reality multiple columns can be included in the ORDER BY clause. 

Possible ideas for the Spider task:
1. Build a diaglog interface that allows the system to interact with human to generate SQL queries that best suit human needs.
2. Since queries are marked with difficulties, Spider can be used to compare the performance discrepancy of a particular model on queries of different difficulties. Or similarly, by comparing different models, Spider can be used to suggest the particular set of features that results in good/bad performance on queries of a specific level of difficulty.

## Assignment Part 3: SQLNet with Torchtext - 35 Points


In the last section, we did data preprocessing, numericalize and embedding, data loading and batching all by ourselves. These steps could be painful and tedious for many NLP tasks. You don't want to restart and write new code for a similar task. It's also very easy to make some small bugs and ruin all your work.

In this part, let's try Torchtext to standardize the text-processing steps. For simplicity, we only demonstrate this transformation on the condition module.

#### 1. Data Prepropessing

In [0]:
#READCODE! Only go over the condition part, SCAN others
def process_one(sql, tables):
    '''
    modified process function for torchtext, basically combine old process 
    and to_batch_seq functions
    '''
    sql_one = {}
    
    # add query metadata
    sql_one['question'] = sql['question']
    sql_one['question_tok'] = sql['question_toks']
    sql_one['query'] = sql['query']
    sql_one['query_tok'] = sql['query_toks']
    sql_one['table_id'] = sql['db_id']
    table = tables[sql['db_id']]
    sql_one['col_org'] = table['column_names_original']
    sql_one['table_org'] = table['table_names_original']
    sql_one['foreign_keys'] = table['foreign_keys']
    column_names = [x.split(" ") for x in [col[1] for col in table['column_names']]]
    sql_one['col_len'] = [len(x) for x in column_names]
    sql_one['col_seq'] = [str(x) for col in column_names for x in col]
    

    # process agg/sel
    sql_one['agg'] = []
    sql_one['sel'] = []
    gt_sel = sql['sql']['select'][1]
    if len(gt_sel) > 3:
        gt_sel = gt_sel[:3]
    for tup in gt_sel:
        sql_one['agg'].append(tup[0])
        sql_one['sel'].append(tup[1][1][1]) #GOLD for sel and agg
    sql_one["sel_num"] = len(set(sql_one['sel']))

    # process where conditions and conjuctions
    sql_one['cond'] = []
    gt_cond = sql['sql']['where']
    if len(gt_cond) > 0:
        conds = [gt_cond[x] for x in range(len(gt_cond)) if x % 2 == 0]
        for cond in conds:
            curr_cond = []
            curr_cond.append(cond[2][1][1])
            curr_cond.append(cond[1])
            if cond[4] is not None:
                curr_cond.append([cond[3], cond[4]])
            else:
                curr_cond.append(cond[3])
            sql_one['cond'].append(curr_cond) #GOLD for COND [[col, op],[]]

    sql_one['cond_num'] = len(sql_one['cond'])
    sql_one['cond_col'] = [x[0] for x in sql_one['cond']]
    sql_one['cond_op'] = [x[1] for x in sql_one['cond']]
    sql_one['gt_cond'] = [x for x in sql_one['cond']]
    sql_one['conj'] = [gt_cond[x] for x in range(len(gt_cond)) if x % 2 == 1]

    # process group by / having
    sql_one['group'] = [x[1] for x in sql['sql']['groupBy']] #assume only one groupby
    having_cond = []
    if len(sql['sql']['having']) > 0:
        gt_having = sql['sql']['having'][0] # currently only do first having condition
        having_cond.append([gt_having[2][1][0]]) # aggregator
        having_cond.append([gt_having[2][1][1]]) # column
        having_cond.append([gt_having[1]]) # operator
        if gt_having[4] is not None:
            having_cond.append([gt_having[3], gt_having[4]])
        else:
            having_cond.append(gt_having[3])
    else:
        having_cond = [[], [], []]
    sql_one['group'].append(having_cond) #GOLD for GROUP [[col1, col2, [agg, col, op]], [col, []]]
    sql_one["group_col"] = sql_one['group'][:-1]
    sql_one["group_num"] = len(sql_one['group']) - 1
    sql_one["having_agg"] = sql_one['group'][-1][0]
    sql_one["having_col"] = sql_one['group'][-1][1]
    sql_one["having_op"] = sql_one['group'][-1][2]

    # process order by / limit
    order_aggs = []
    order_cols = []
    sql_one['order'] = []
    order_par = 4
    gt_order = sql['sql']['orderBy']
    limit = sql['sql']['limit']
    if len(gt_order) > 0:
        order_aggs = [x[1][0] for x in gt_order[1][:1]] # limit to 1 order by
        order_cols = [x[1][1] for x in gt_order[1][:1]]
        if limit != None:
            if gt_order[0] == 'asc':
                order_par = 0
            else:
                order_par = 1
        else:
            if gt_order[0] == 'asc':
                order_par = 2
            else:
                order_par = 3

    sql_one['order'] = [order_aggs, order_cols, order_par] #GOLD for ORDER [[[agg], [col], [dat]], []]
    sql_one["order_agg"] = sql_one['order'][0]
    sql_one["order_col"] = sql_one['order'][1]
    sql_one["order_num"] = len(sql_one['order'][1])
    sql_one["order_parity"] = sql_one['order'][2]

    # process intersect/except/union
    sql_one['special'] = 0
    if sql['sql']['intersect'] is not None:
        sql_one['special'] = 1
    elif sql['sql']['except'] is not None:
        sql_one['special'] = 2
    elif sql['sql']['union'] is not None:
        sql_one['special'] = 3
        
    return sql_one
  

In [0]:
def process(sql_path, table_data):
    '''
    process all input data
    '''
    with open(sql_path) as inf:
        sql_data = json.load(inf)
        
    tables = {}
    for i in range(len(table_data)):
        table = table_data[i]
        db_name = table['db_id']
        tables[db_name] = table

    output_sql = []
    for i in range(len(sql_data)):
        sql = sql_data[i]
        sql_one = process_one(sql, tables)
        output_sql.append(sql_one)

    return output_sql

#### 2. Torchtext Fields and Examples

Torchtext `Field`s  are used to define how you want the data preprocessed. The `Example` object bundles the attributes of a single data point together.

**Note**: Please follow [toturial 1](http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/), [toturial 2](http://anie.me/On-Torchtext/) or the official [doc](https://torchtext.readthedocs.io/en/latest/data.html#) and [code](https://github.com/pytorch/text) to understand how Torchtext works.

In [0]:
#READCODE!
def get_fields():
    '''
    fields (inputs: question_tok, col_seq, col_len, and targets: cond_col, cond_op) for cond module
    '''
    fields = {}
    fields["question_tok"] = torchtext.data.Field(include_lengths=True, lower=True, batch_first=True)
    fields["col_seq"] = torchtext.data.Field(include_lengths=True, lower=True, batch_first=True)
    fields["col_len"] = torchtext.data.Field(use_vocab=False, pad_token=-1, batch_first=True)
    fields["cond_col"] = torchtext.data.Field(use_vocab=False,  pad_token=-1, include_lengths=True, batch_first=True)
    fields["cond_op"] = torchtext.data.Field(use_vocab=False,  pad_token=-1, batch_first=True)
    fields["indices"] = torchtext.data.Field(use_vocab=False, sequential=False, batch_first=True)
    
    field_list = [(k, fields[k]) for k in fields.keys()]
    
    return field_list

In [0]:
#READCODE!
def construct_examples(data, fields):
    '''
    convert each data point into a example object in torchtext
    '''
    keys = [k[0] for k in fields if k[0] != 'indices']
    data_cond = [dict((k, d[k]) for k in keys) for d in data]
    examples = []
    for i, ex in enumerate(data_cond):
        # yield torchtext.data.Example.fromlist([ex[k] for k in keys] + [i], fields)
        examples.append(torchtext.data.Example.fromlist([ex[k] for k in keys] + [i], fields))

    return examples

#### 3. Vocab

Build and merge vocab for question and column name (`question_tok` and `col_seq` Fields) tokens:

In [0]:
def filter_counter(freqs, min_freq):
    cnt = Counter()
    for k, v in freqs.items():
        if (min_freq is None) or (v >= min_freq):
            cnt[k] = v
    return cnt
  
def merge_vocabs(vocabs, min_freq=0, vocab_size=None):
    """
    Merge individual vocabularies (assumed to be generated from disjoint
    documents) into a larger vocabulary.

    Args:
        vocabs: `torchtext.vocab.Vocab` vocabularies to be merged
        vocab_size: `int` the final vocabulary size. `None` for no limit.
    Return:
        `torchtext.vocab.Vocab`
    """
    merged = Counter()
    for vocab in vocabs:
        merged += filter_counter(vocab.freqs, min_freq)
    return torchtext.vocab.Vocab(merged,max_size=vocab_size, min_freq=min_freq)

#READCODE!
def build_vocab(train, dev, max_size, min_freq):
    '''
    build torchtext vocab for question_tok and col_seq fields, 
    and merge vocabs of the two
    '''
    fields = train.fields

    merge_list = []
    merge_name_list = ("question_tok", "col_seq")
    for split in (dev, train,):
        for merge_name_it in merge_name_list:
            fields[merge_name_it].build_vocab(
                split, max_size=max_size, min_freq=min_freq)
            merge_list.append(fields[merge_name_it].vocab)

    # need to know all the words to filter the pretrained word embeddings
    merged_vocab = merge_vocabs(merge_list, vocab_size=max_size)
    for merge_name_it in merge_name_list:
        fields[merge_name_it].vocab = merged_vocab
        
    return merged_vocab

#### 4. Load the data and create data iterators

Torchtext then passes the Dataset to an Iterator. Iterators handle numericalizing, batching, packaging, and moving the data to the GPU. 

In [0]:
#READCODE!
def load_dataset(dataset_dir):
    '''
    load and process data into torchtext dataset, 
    build vocab and iterators
    '''
    with open(TABLE_PATH) as inf:
        print("loading data from %s"%TABLE_PATH)
        table_data= json.load(inf)
    
    print("processing train and dev data...")
    train_data = process(TRAIN_PATH, table_data)
    valid_data = process(DEV_PATH, table_data)
    
    print("building fields...")
    fields = get_fields()
    
    print("constructing examples...")
    train_examples = construct_examples(train_data, fields)
    valid_examples = construct_examples(valid_data, fields)
    
    print("creating torchtext dataset...")
    valid = torchtext.data.Dataset(valid_examples, fields)
    train = torchtext.data.Dataset(train_examples, fields)
    
    print("building vocab...")
    vocab = build_vocab(train, valid, max_size=50000, min_freq=1)
    
    print("creating iterators for train and dev...")
    train_iter, valid_iter = torchtext.data.Iterator.splits(
        (train, valid), sort_key=lambda x: len(x.question_tok),
        batch_sizes=(64, 64), device=device)
    
    print("done!")
      
    return train_iter, valid_iter, vocab

In [26]:
train_iter, valid_iter, vocab = load_dataset("data/")

loading data from /content/gdrive/My Drive/nlp_hw4/data/tables.json
processing train and dev data...
building fields...
constructing examples...
creating torchtext dataset...
building vocab...
creating iterators for train and dev...
done!


#### 5. Reconstruct inputs for the model

Convert standardized Torchtext batched inputs to the inputs for SQLNet.

**TODO**: reconstruct target label inputs for the condition module in SQLNet. You need to understand what the Torchtext standardized inputs look like.

In [0]:
#READCODE!
#TODO!
def reconstruct_input(cond_op, cond_col, col_concat, col_len):
    '''
    function to reconstruct column and target label input for SQLNet with torchtext
    please check epoch_train function below to see where it gets called
    
    ---Parameters---
    
    cond_op (tensor): shape: (batch size, max where condition number in currect batch)
           if no operation or condition, -1 padded, otherwise indices of
           ('not', 'between', '=', '>', '<', '>=', '<=', '!=', 'in', 'like', 'is', 'exists')
    cond_col (tensor): shape: (batch size, max where condition number in currect batch)
           if no condition column, -1 padded, otherwise indices of columns
    '''
    #reconstruct column inputs for SQLNet
    col_concat_list = col_concat.data.tolist()
    col_len_list = col_len.data.tolist()
    
    col_name_len = [l for ls in col_len_list for l in ls if l != -1]
    col_num = [len([l for l in ls if l != -1]) for ls in col_len_list]
    col_B = len(col_name_len)
    max_len = max(col_name_len)
    col_emb_inds = np.zeros((col_B, max_len), dtype=np.int64)
    cols_all = []
    for col_concat_l, col_len_l in zip(col_concat_list, col_len_list):
        start_ind = 0
        end_ind = 0
        for ll in col_len_l:
            if ll == -1:
                break
            end_ind += ll
            cols_all.append(col_concat_l[start_ind:end_ind])
            start_ind += ll
    
    for i in range(col_B):
        col_emb_inds[i,:len(cols_all[i])] = cols_all[i]
    #vocab indices of words in all column names
    col_ind_inp = torch.from_numpy(col_emb_inds).to(device)
    
    #TODO:IMPLEMENT YOUR CODE BELOW
    #reconstruct target label inputs for SQLNet 
    #here for simplicity, we only do it for cond
    #except cond, others are fake numbers: 
    #one example in ans_seq: [0, 1, cond_num, cond_col, cond_op, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    ans_seq = []
    cond_op_list = cond_op.data.tolist()
    cond_col_list = cond_col.data.tolist()
    for cond_op_l, cond_col_l in zip(cond_op_list, cond_col_list):
      cond_op_cur = []
      cond_col_cur = []
      for l1 in cond_op_l:
        if l1 != -1:
          cond_op_cur.append(l1)
      for l2 in cond_col_l:
        if l2 != -1:
          cond_col_cur.append(l2)
      ans_seq.append([0, 1, len(cond_col_cur), cond_col_cur, cond_op_cur, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
      
    return col_ind_inp, col_name_len, col_num, ans_seq

In [0]:
## you can uncomment this to see what a iterator looks like
# for i, batch in enumerate(valid_iter):
#     q, q_len = batch.question_tok
#     col_concat, col_concat_len = batch.col_seq
#     col_len = batch.col_len
#     cond_op = batch.cond_op
#     cond_col, cond_num = batch.cond_col
    
#     x_len = q_len.data.tolist()
#     col_ind_inp, col_name_len, col_num, ans_seq= reconstruct_input(cond_op, cond_col, col_concat, col_len)

#### 6. Model training and testing

Most of code is similar to the code in the last section but with modifications to accommodate the Torchtext data processing pipeline. 

In [0]:
#SCAN
class SQLNet(nn.Module):
    '''
    modified SQLNet with only cond module for demo
    '''
    def __init__(self, vocab, N_word, N_h=120, N_depth=2):
        super(SQLNet, self).__init__()
        self.N_h = N_h
        self.N_depth = N_depth

        self.max_col_num = 45
        self.max_tok_num = 200
        self.SQL_TOK = ['<UNK>', '<END>', 'WHERE', 'AND',
                'EQL', 'GT', 'LT', '<BEG>']
        self.COND_OPS = ['EQL', 'GT', 'LT']
        
        #MODIFIED: self embedding layer -> nn embedding layer
#         vocab.load_vectors("glove.6B.50d")
        self.embed_layer = nn.Embedding(len(vocab), N_word)
#         self.embed_layer.weight.data.copy_(vocab.vectors)

        #Predict where condition
        self.cond_pred = CondPredictor(N_word, N_h, N_depth)

        self.CE = nn.CrossEntropyLoss()
        self.softmax = nn.Softmax(dim=-1)
        self.sigm = nn.Sigmoid()

    def forward(self, q_inds, q_len, col_inds, col_name_len, col_num,
            gt_where=None, gt_cond=None, gt_sel=None):
        
        B = len(q_len)

        sel_score = None
        cond_score = None
        group_score = None
        order_score = None

        q_emb_var = self.embed_layer(q_inds)
        col_inp_var = self.embed_layer(col_inds)

        cond_score = self.cond_pred(q_emb_var, q_len, col_inp_var, col_num, col_name_len, gt_cond=gt_cond)

        return cond_score


    def loss(self, score, truth_num):

        cond_score = score

        cond_num_score, cond_col_score, cond_op_score = cond_score

        B = len(truth_num)
        loss = 0
        #----------------loss for cond_pred--------------------#
        #cond_num_score, cond_col_score, cond_op_score = cond_score

        #Evaluate the number of conditions
        cond_num_truth = list(map(lambda x:x[2], truth_num))
        cond_num_truth_var = torch.from_numpy(np.array(cond_num_truth)).to(device)
        loss += self.CE(cond_num_score, cond_num_truth_var)
        #Evaluate the columns of conditions
        T = len(cond_col_score[0])
        truth_prob = np.zeros((B, T), dtype=np.float32)
        for b in range(B):
            if len(truth_num[b][3]) > 0:
                truth_prob[b][list(truth_num[b][3])] = 1

        cond_col_truth_var = torch.from_numpy(truth_prob).to(device)
        cond_col_prob = self.sigm(cond_col_score)
        bce_loss = -torch.mean( 3*(cond_col_truth_var * \
                torch.log(cond_col_prob+1e-10)) + \
                (1-cond_col_truth_var) * torch.log(1-cond_col_prob+1e-10) )
        loss += bce_loss
        #Evaluate the operator of conditions
        for b in range(len(truth_num)):
            if len(truth_num[b][4]) == 0:
                continue
            cond_op_truth_var = torch.from_numpy(np.array(truth_num[b][4])).to(device)
            cond_op_pred = cond_op_score[b, :len(truth_num[b][4])]
            loss += (self.CE(cond_op_pred, cond_op_truth_var) \
                    / len(truth_num))

        return loss


    def check_acc(self, pred_queries, gt_queries):
        B = len(gt_queries)

        tot_err = 0.0
        cond_err = cond_num_err = cond_col_err = cond_op_err = 0.0
        for b, pred_qry in enumerate(pred_queries):

            good = True
            tot_flag = True
            cond_flag = True

            # conds
            cond_pred = pred_qry['conds']
            flag = True
            if len(cond_pred) != gt_queries[b][2]:
                flag = False
                cond_num_err += 1
                cond_flag = False
            if flag and set(x[0] for x in cond_pred) != set(gt_queries[b][3]):
                flag = False
                cond_col_err += 1
                cond_flag = False
            for idx in range(len(cond_pred)):
                if not flag:
                    break
                gt_idx = gt_queries[b][3].index(cond_pred[idx][0])
                if flag and gt_queries[b][4][gt_idx] != cond_pred[idx][1]:
                    flag = False
                    cond_op_err += 1
                    cond_flag = False

            if not cond_flag:
                cond_err += 1
                good = False

            if not good:
                tot_err += 1

        return np.array((0, cond_err, 0, 0)), tot_err


    def gen_query(self, score):

        cond_score = score

        cond_num_score, cond_col_score, cond_op_score = [x.data.cpu().numpy() if x is not None else None for x in cond_score]
        ret_queries = []
        B = len(cond_num_score)
        for b in range(B):
            cur_query = {}
            #---------get cond predict
            cur_query['conds'] = []
            cond_num = np.argmax(cond_num_score[b])
            max_idxes = np.argsort(-cond_col_score[b])[:cond_num]
            for idx in range(cond_num):
                cur_cond = []
                cur_cond.append(max_idxes[idx])
                cur_cond.append(np.argmax(cond_op_score[b][idx]))
                cur_query['conds'].append(cur_cond)
            ret_queries.append(cur_query)

        return ret_queries

Torchtext handles data loading, processing, padding, and batching for you. 

**TODO**: Let's modify `epoch_train`  to use the Torchtext iterator object to batch the data and then train the model:

In [0]:
N_word = 50
model = SQLNet(vocab, N_word=N_word).to(device)

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)

#READCODE!
#TODO!
def epoch_train(model, optimizer, data_iter):
    model.train()
    total_loss = 0.0
    count = 0
    #TODO:IMPLEMENT YOUR CODE BELOW
    #iterate over data
    for i, batch in enumerate(data_iter):
        #1. reconstruct inputs for SQLNet
        q_inds, q_len = batch.question_tok
        col_concat, col_concat_len = batch.col_seq
        col_len = batch.col_len
        cond_op = batch.cond_op
        cond_col, cond_num = batch.cond_col
        
        x_len = np.array(q_len.data.tolist())
        col_inds, col_name_len, col_num, ans_seq = reconstruct_input(cond_op, cond_col, col_concat, col_len)
        col_name_len = np.array(col_name_len)
        #2. run SQLNet forward pass (ingore gt_cond input for model.forward)
        score = model.forward(q_inds, x_len, col_inds, col_name_len, col_num)
        #3. compute loss, add it to total loss
        loss = model.loss(score, ans_seq)
        total_loss += loss.item()
        #4. update weights in SQLNet
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        count += 1
      
    return total_loss / count


def epoch_acc(model, data_iter):
    model.eval()
    one_acc_num = 0.0
    tot_acc_num = 0.0
    count = 0
    for i, batch in enumerate(data_iter):
        q_inds, q_len = batch.question_tok
        col_concat, col_concat_len = batch.col_seq
        col_len = batch.col_len
        cond_op = batch.cond_op
        cond_col, cond_num = batch.cond_col

        x_len = np.array(q_len.data.tolist())
        col_inds, col_name_len, col_num, ans_seq = reconstruct_input(cond_op, cond_col, col_concat, col_len)
        col_name_len = np.array(col_name_len)
        score = model.forward(q_inds, x_len, col_inds, col_name_len, col_num)
        pred_queries = model.gen_query(score)
        one_err, tot_err = model.check_acc(pred_queries, ans_seq)
        one_acc_num += (len(ans_seq)-one_err)
        tot_acc_num += (len(ans_seq)-tot_err)
        
        count += len(ans_seq)
        
    return tot_acc_num / count, one_acc_num / count

Let's run train and evaluate model:

In [31]:
BATCH_SIZE=64
#initial accuracy
init_acc = epoch_acc(model, valid_iter)
best_sel_acc = init_acc[1][0]
best_cond_acc = init_acc[1][1]
best_group_acc = init_acc[1][2]
best_order_acc = init_acc[1][3]
best_tot_acc = 0.0

for i in range(30):
    print('Epoch %d @ %s'%(i+1, datetime.datetime.now()))
    print(' Loss = %s'%epoch_train(model, optimizer, train_iter))
    train_tot_acc, train_bkd_acc = epoch_acc(model, train_iter)
    print(' Train acc_qm: %s' % train_tot_acc)
    print(' Breakdown results: sel: %s, cond: %s, group: %s, order: %s'\
        % (train_bkd_acc[0], train_bkd_acc[1], train_bkd_acc[2], train_bkd_acc[3]))

    val_tot_acc, val_bkd_acc = epoch_acc(model, valid_iter)
    print(' Dev acc_qm: %s' % val_tot_acc)
    print(' Breakdown results: sel: %s, cond: %s, group: %s, order: %s'\
        % (val_bkd_acc[0], val_bkd_acc[1], val_bkd_acc[2], val_bkd_acc[3]))

    #save models
    if val_bkd_acc[0] > best_sel_acc:
        best_sel_acc = val_bkd_acc[0]
        print("Saving sel model...")
        torch.save(model.sel_pred.state_dict(), os.path.join(SAVED_MODEL_DIR, "sel_models.dump"))
    if val_bkd_acc[1] > best_cond_acc:
        best_cond_acc = val_bkd_acc[1]
        print("Saving cond model...")
        torch.save(model.cond_pred.state_dict(), os.path.join(SAVED_MODEL_DIR, "cond_models.dump"))
    if val_bkd_acc[2] > best_group_acc:
        best_group_acc = val_bkd_acc[2]
        print("Saving group model...") 
        torch.save(model.group_pred.state_dict(), os.path.join(SAVED_MODEL_DIR, "group_models.dump"))
    if val_bkd_acc[3] > best_order_acc:
        best_order_acc = val_bkd_acc[3]
        print("Saving order model...")
        torch.save(model.order_pred.state_dict(), os.path.join(SAVED_MODEL_DIR, "order_models.dump"))
    if val_tot_acc > best_tot_acc:
        best_tot_acc = val_tot_acc

    print(' Best val sel = %s, cond = %s, group = %s, order = %s, tot = %s'%(best_sel_acc, best_cond_acc, best_group_acc, best_order_acc, best_tot_acc))

Epoch 1 @ 2019-04-09 23:06:53.816101
 Loss = 1.6299720948392695
 Train acc_qm: 0.451
 Breakdown results: sel: 1.0, cond: 0.451, group: 1.0, order: 1.0
 Dev acc_qm: 0.4284332688588008
 Breakdown results: sel: 1.0, cond: 0.4284332688588008, group: 1.0, order: 1.0
Saving cond model...
 Best val sel = 1.0, cond = 0.4284332688588008, group = 1.0, order = 1.0, tot = 0.4284332688588008
Epoch 2 @ 2019-04-09 23:07:29.315893
 Loss = 1.2228886967355554
 Train acc_qm: 0.4677142857142857
 Breakdown results: sel: 1.0, cond: 0.4677142857142857, group: 1.0, order: 1.0
 Dev acc_qm: 0.45551257253384914
 Breakdown results: sel: 1.0, cond: 0.45551257253384914, group: 1.0, order: 1.0
Saving cond model...
 Best val sel = 1.0, cond = 0.45551257253384914, group = 1.0, order = 1.0, tot = 0.45551257253384914
Epoch 3 @ 2019-04-09 23:08:05.989370
 Loss = 0.9800558380105279
 Train acc_qm: 0.5031428571428571
 Breakdown results: sel: 1.0, cond: 0.5031428571428571, group: 1.0, order: 1.0
 Dev acc_qm: 0.48839458413926

## Submission

Now that you have completed the assignment, follow the steps below to submit your aissgnment:
1. Click __Runtime__  > __Run all__ to generate the output for all cells in the notebook.
2. Save the notebook with the output from all the cells in the notebook by click __File__ > __Download .ipynb__.
3. Copy model train and test prints, answers to all short questions, and the shareable line of this notebook to a `README.txt` file.
4. Put the .ipynb file and `README.txt` under your hidden directory on the Zoo server `~/hidden/<YOUR_PIN>/Homework4/`.
5. As a final step, run a script that will set up the permissions to your homework files, so we can access and run your code to grade it. Make sure the command be;pw runs without errors, and do not make any changes or run the code again. If you do run the code again or make any changes, you need to run the permissions script again. Submissions without the correct permissions may incur some grading penalty.
`/home/classes/cs477/bash_files/hw4_set_permissions.sh <YOUR_PIN>`